# Load modules and external files

You need to import four python scripts for implied volatility calibration :
- *newton.py*
- *BSImplVol.py*
- *BS.py*
- *Bisect.py*

In [0]:
import pandas as pd
import numpy as np
import scipy.integrate as integrate
from scipy import interpolate
import math
import matplotlib.pyplot as plt
import tensorflow as tf

plt.style.use('ggplot')
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import sklearn as skl
from sklearn import preprocessing
import importlib
import scipy.stats as st
import numpy as np
import math
import scipy.stats as st
import matplotlib.ticker as mtick
import time
from scipy import interpolate

In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
#Load python files to google colaborative environment
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from BS import bsformula
from Bisect import bisect
from newton import newton
from BSImplVol import bsimpvol

# Load data with google colab 

You will find in github repository six days of data.
For each day you need to load six csv files :
- *underlying.csv* for the stock value.
- *locvol.csv* for the local volatility calibrated with tree pricing and tikhonov volatility (see Crépey (2002)).
- *dividend.csv* for dividend extracted from put-call parity.
- *discount.csv* for zero-coupon curve. 
- *dataTrain.csv* for prices and/or implied volatility used in training set.
- *dataTest.csv* for prices and/or implied volatility used in testing set.

In [0]:
#Load csv files to get data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

In [0]:
#Read csv files as dataFrames
zeroCouponCurve = pd.read_csv("discount.csv",decimal=".").apply(pd.to_numeric)
dividendCurve = pd.read_csv("dividend.csv",decimal=".").apply(pd.to_numeric)
trainingData = pd.read_csv("dataTrain.csv",decimal=".").apply(pd.to_numeric)
testingData = pd.read_csv("dataTest.csv",decimal=".").apply(pd.to_numeric)
underlyingNative = pd.read_csv("underlying.csv",decimal=".").apply(pd.to_numeric)
localVolatilityNative = pd.read_csv("locvol.csv",decimal=".").apply(pd.to_numeric)


In [0]:
#Format dividend curve as a Pandas series
dividendDf = dividendCurve.set_index('Maturity').sort_index()
dividendDf.loc[1.0] = 0.0
dividendDf.sort_index(inplace=True)
dividendDf.tail()

In [0]:
#Format zero coupon curve as a Pandas series
rateCurveDf = zeroCouponCurve.set_index('Maturity').sort_index()
# keep only rates expriring before 1 year
rateCurveDf = rateCurveDf.loc[rateCurveDf.index <= 1.01]
rateCurveDf.head()

In [0]:
#Format local volatility
localVolatility = localVolatilityNative.dropna()
localVolatility["Strike"] = localVolatility["stock(%)"] * underlyingNative["S"].values
localVolatility["date"] = localVolatility["date"].round(decimals=3)
renameDict = {"date": "Maturity", 
              "vol" : "LocalVolatility", 
              "stock(%)" : "StrikePercentage"}
localVolatility = localVolatility.rename(columns=renameDict).set_index(["Strike", "Maturity"])
localVolatility.head()

In [0]:
localVolatility.head()

In [0]:
underlyingNative.head()

In [0]:
testingData.head()

In [0]:
#Treatment for training data
filteredTestingData = testingData[(testingData["Implied vol."] > 0) * (testingData["Option price"] > 0)]
filteredTestingData["Maturity"] = filteredTestingData["Maturity"].round(decimals=3)
renameDict = {"Implied vol.": "ImpliedVol", 
              "Option price" : "Price", 
              "Implied delta" : "ImpliedDelta", 
              "Implied gamma" : "ImpliedGamma",
              "Implied theta" : "ImpliedTheta",
              "Local delta" : "LocalDelta",
              "Local gamma" : "LocalGamma"}
formattedTestingData = filteredTestingData.rename(columns=renameDict).set_index(["Strike", "Maturity"])["ImpliedVol"]
formattedTestingData.head()

In [0]:
trainingData.head()

In [0]:
#Treatment for testing data
filteredTrainingData = trainingData[(trainingData["Calibrated\nvol."] > 0) * (trainingData["Option\nprice"] > 0) * (trainingData["Option\ntype"] == 2)]
filteredTrainingData["Maturity"] = filteredTrainingData["Maturity"].round(decimals=3)
renameDict = {"Option\ntype" : "OptionType", 
              "Option\nprice" : "Price", 
              "Calibrated\nvol." : "ImpliedVol",#"LocalImpliedVol", 
              "Implied\nvol." : "LocalImpliedVol"}#"ImpliedVol"}
formattedTrainingData = filteredTrainingData.drop(["Active", "Market vol. -\nCalibrated vol."],axis=1).rename(columns=renameDict).set_index(["Strike","Maturity"])
formattedTrainingData.head()

In [0]:
formattedTrainingData.shape

# Formatting data

### Boostsrapping Rate Curve


- For bootstrapping short rate $r$ and dividend rate $q$, we assume piecewise constant short rate for risk free rate and dividend i.e. 
$\exp{(-\int_{0}^{T} r_t d_t)} = \exp{(-\sum_{i} r_i h)}$ and $\exp{(\int_{0}^{T} q_t d_t)} = \exp{(\sum_{i} q_i h)}$.
- $\forall i \in \{0,..,N\}$ with $ t_0 = 0$ and $t_N = T$, we have that $\frac{\log{B(0,t_{i+1})} - \log{B(0,t_i)}}{h} = r_i$ with $B(0,T_i)$ the price of a bond expiring at time $t_i$. 
- For dividend, we just to substitute $B(0,T_i)$ with with spot action price plus dividend cash flow received until time $T_i$ i.e. $S_{t_0} + \sum\limits_i Div_{t_i}$.
- Then we linearly interpolate $r$ and $q$.
-  Linear interpolation is also used for integrals $\int_{0}^{T} q_t d_t$ and $\int_{0}^{T} r_t d_t$ in order to obtain discount factor or dividend factor. 

In [0]:
#Compute the integral and return the linear interpolation function 
def interpIntegral(curve):
    #curve is piece-wise constant
    timeDelta = curve.index.to_series().diff().fillna(0)
    timeStep = np.linspace(0,0.99,100)
    integralStepWise = (curve * timeDelta).cumsum()
    integralStepWise.loc[0] = 0.0
    integralStepWise.sort_index(inplace=True)
    integralSpline = interpolate.interp1d(integralStepWise.index,
                                          integralStepWise, 
                                          fill_value= 'extrapolate', 
                                          kind ='linear')
    return pd.Series(integralSpline(timeStep),index=timeStep), integralSpline

def bootstrapZeroCoupon(curvePrice, name):
    #Bootstrap short rate curve
    def computeShortRate(curve) :
      shortRateList = [] 
      for i in range(curve.size):
        if i == 0 :
          shortRateList.append(-(np.log(curve.iloc[i]))/(curve.index[i]))
        else : 
          shortRateList.append(-(np.log(curve.iloc[i])-np.log(curve.iloc[i-1]))/(curve.index[i]-curve.index[i-1]))
      return pd.Series(shortRateList,index = curve.index)
    #For t=0 we take the first available point to ensure right continuity
    riskFreeCurve = computeShortRate(curvePrice)
    riskFreeCurve.loc[0.00] = riskFreeCurve.iloc[0]
    riskFreeCurve = riskFreeCurve.sort_index()

    #Bootstrap yield curve
    def zeroYield(x):
      if(float(x.name) < 1):
        return (1/x - 1)/float(x.name)
      else:
        return (x**(-1/float(x.name)) - 1)
    yieldCurve = curvePrice.apply(zeroYield, axis = 1)
    yieldCurve.loc[0.00] = yieldCurve.iloc[0]
    yieldCurve = yieldCurve.sort_index()

    plt.plot(riskFreeCurve, label = "Short rate")

    #Interpolate short rate curve and yield curve
    timeStep = np.linspace(0,0.99,100)
    riskCurvespline = interpolate.interp1d(riskFreeCurve.index,
                                           riskFreeCurve,#riskFreeCurve[name],
                                           fill_value= 'extrapolate',
                                           kind ='next')
    interpolatedCurve = pd.Series(riskCurvespline(timeStep),index=timeStep)
    plt.plot(interpolatedCurve, label="Interpolated short rate")
    plt.legend()
    plt.show()

    plt.plot(yieldCurve, label = "Yield curve")
    yieldCurvespline = interpolate.interp1d(yieldCurve.index,
                                            yieldCurve['Price'],
                                            fill_value= 'extrapolate',
                                            kind ='next')
    interpolatedCurve = pd.Series(yieldCurvespline(timeStep),index=timeStep)
    plt.plot(interpolatedCurve, label = "Interpolated Yield curve")
    plt.legend()
    plt.show()
    
    #Integrate short rate
    interpolatedIntegral, riskFreeIntegral = interpIntegral(riskFreeCurve)
    plt.plot(interpolatedIntegral)
    plt.show()

    return riskFreeCurve, riskCurvespline, yieldCurve, yieldCurvespline, interpolatedIntegral, riskFreeIntegral


In [0]:
riskFreeCurve, riskCurvespline, yieldCurve, yieldCurvespline, interpolatedIntegral, riskFreeIntegral = bootstrapZeroCoupon(rateCurveDf, "Short rate")

In [0]:
riskFreeCurve

In [0]:
interpolatedIntegral

### Boostraping dividend curve

In [0]:
def bootstrapDividend(curvePrice, underlying, name):
    #Compute cumulative sum of dividend plus spot price
    priceEvolution = underlying['S'].iloc[0] - curvePrice['Amount'].cumsum()
    priceEvolution.loc[0] = underlying['S'].iloc[0]
    priceEvolution.sort_index(inplace=True)

    #Bootstrap short rate for dividend
    def computeShortRate(curve) :
      shortRateList = [] 
      for i in range(curve.size):
        if i == 0 :
          shortRateList.append(-(np.log(curve.iloc[i+1])-np.log(curve.iloc[i]))/(curve.index[i+1]-curve.index[i]))
        else : 
          shortRateList.append(-(np.log(curve.iloc[i])-np.log(curve.iloc[i-1]))/(curve.index[i]-curve.index[i-1]))
      return pd.Series(shortRateList,index = curve.index).dropna()
    logReturnDividendDf = computeShortRate(priceEvolution)

    #Dividend yield curve
    def divYield(x):
      return ((priceEvolution[x]/priceEvolution.iloc[0])**(1/float(x)) - 1) #np.log(priceEvolution[x]/priceEvolution.iloc[0])/x
    dividendYield = logReturnDividendDf.index.to_series().tail(-1).apply(divYield)
    dividendYield.loc[0.00] = dividendYield.iloc[0]
    dividendYield = dividendYield.sort_index()

    plt.plot(logReturnDividendDf, label = "Short rate")

    #Interpolate short rate curve and yield curve
    timeStep = np.linspace(0,0.99,100)
    logReturnDividendSpline = interpolate.interp1d(logReturnDividendDf.index,
                                                   logReturnDividendDf,#logReturnDividendDf[name],
                                                   fill_value= 'extrapolate',
                                                   kind ='next')
    interpolatedCurve = pd.Series(logReturnDividendSpline(timeStep),index=timeStep)
    plt.plot(interpolatedCurve, label="Interpolated short rate")
    plt.legend()
    plt.show()

    plt.plot(dividendYield, label = "Yield curve")
    yieldCurvespline = interpolate.interp1d(dividendYield.index,
                                            dividendYield.values,
                                            fill_value= 'extrapolate',
                                            kind ='next')
    interpolatedCurve = pd.Series(yieldCurvespline(timeStep),index=timeStep)
    plt.plot(interpolatedCurve, label = "Interpolated Yield curve")
    plt.legend()
    plt.show()
    
    #Integrate short rate
    interpolatedIntegral, logReturnDividendIntegral = interpIntegral(logReturnDividendDf)#logReturnDividendDf[name])
    plt.plot(interpolatedIntegral)
    plt.show()

    return logReturnDividendDf, logReturnDividendSpline, dividendYield, yieldCurvespline, interpolatedIntegral, logReturnDividendIntegral

In [0]:
spreadDividend, divSpline, yieldDividend, divYieldSpline, interpolatedIntegral, divSpreadIntegral  = bootstrapDividend(dividendDf, underlyingNative, "Spread")

In [0]:
spreadDividend

In [0]:
interpolatedIntegral

### Pricing black-scholes price

#### Change of variable

- In presence of dividend rate $d$ and risk free rate $r$ Dupire formula is :   $$\sigma^2(T,K) = 2 \frac{ \partial_T P(T,K) + (r-q)\partial_K P(T,K) + qP(T,K)}{K² \partial_{K}^2 P(T,K)}$$ 
with Strike $K$, Maturity $T$, dividend rate $q$ and risk-free rate $r$, $P$ our pricing function. 
- We apply the following change of variable : $$ w(T,k) = \exp{(\int_{0}^{T} q_t dt)} P(T,K)$$ with $K = k \exp{(\int_{0}^{T} (r_t - q_t) dt)} $.

- Then Dupire equation becomes :  $\sigma^2(T,K) = 2 \frac{ \partial_T w(T,k)}{k² \partial_{k}^2 w(T,k)}$. 
- If we learn the mapping $v$ with a neural network then we should obtain quickly by adjoint differentiation $\partial_T w$ and $\partial_{k²}^2 w$ and therefore $\sigma$.


In [0]:
import scipy.stats as st
#Density derivative
def dpdf(x):
    v = 1
    return -x*np.exp(-x**2/(2.0*v**2))/(v**3*np.sqrt(2.0*np.pi))
    

def generalizedGreeks(cp, s, k, rf, t, v, div, rfInt, divInt):
        """ Price an option using the Black-Scholes model.
        cp: +1/-1 for call/put
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate at time t
        div: dividend at time t
        rfInt: deterministic risk-free rate integrated between 0 and t
        divInt: deterministic dividend integrated between 0 and t
        """

        d1 = (np.log(s/k)+(rfInt-divInt+0.5*v*v*t))/(v*np.sqrt(t))
        d2 = d1 - v*np.sqrt(t)
        
        Nd1 = st.norm.cdf(cp*d1)
        Nd2 = st.norm.cdf(cp*d2)

        discountFactor = np.exp(-rfInt)
        forwardFactor = np.exp(-divInt)
        avgDiv = divInt/t
        avgRf = rfInt/t

        optprice = (cp*s*forwardFactor*Nd1) - (cp*k*discountFactor*Nd2)

        delta = cp*Nd1
        vega  = s*np.sqrt(t)*st.norm.pdf(d1)
        delta_k = -s*forwardFactor*Nd1/(v*np.sqrt(t)*k) - cp*discountFactor*Nd2 + k*discountFactor*Nd2/(v*np.sqrt(t)*k)
        
        gamma_k = s*forwardFactor/((v*np.sqrt(t)*k)**2)*(Nd1*v*np.sqrt(t) + cp*dpdf(cp*d1)) - k*discountFactor/((v*np.sqrt(t)*k)**2)*(Nd2*v*np.sqrt(t) + cp*dpdf(cp*d2)) +  2.0*discountFactor*Nd2/(v*np.sqrt(t)*k)  

        dd1_dt = (avgRf-avgDiv+0.5*v*v)/(v*np.sqrt(t)) - 0.5*(np.log(s/k)+(rfInt-divInt+0.5*v*v*t))/(v*v*t**(3/2))
        dd2_dt = dd1_dt - 0.5*v/np.sqrt(t)
        delta_T = avgRf*cp*k*discountFactor*Nd2 - avgDiv*cp*s*forwardFactor*Nd1 + s*forwardFactor*Nd1*dd1_dt- k*discountFactor*Nd2*dd2_dt
        
        return optprice, delta, vega, delta_k, gamma_k, delta_T

In [0]:
S0 = underlyingNative["S"].values
#Change of variable for deterministic discount curve and dividend curve
def changeOfVariable(s,t):
  def qInterp(m):
    return divSpreadIntegral(m).astype(np.float32)
  q = qInterp(t)
  
  def rInterp(m):
    return riskFreeIntegral(m).astype(np.float32)
  r = rInterp(t)

  factorPrice = np.exp( - q )

  divSpread = q-r

  factorStrike = np.exp( divSpread )
  adjustedStrike = np.multiply(s, factorStrike)
  return adjustedStrike, factorPrice

#Change of variable for constant discount and dividend short rate 
def changeOfVariable_BS(s,t):
  
  factorPrice = np.exp( - q*t )

  divSpread = (q-r)*t

  factorStrike = np.exp( divSpread )
  adjustedStrike = np.multiply(s, factorStrike)
  return adjustedStrike, factorPrice

#Generate a proper dataset from implied volatility
def generateTestingData(impliedVol, S0, rIntegralSpline, qIntegralSpline, rSpline, qSpline):

  x_train = impliedVol.index.to_frame()
  x_train["ImpliedVol"] = impliedVol
  isPut = True
  cp = -1 if isPut else 1
  impliedPriceFunction = lambda x : generalizedGreeks(cp, 
                                                      S0, 
                                                      x["Strike"] , 
                                                      rSpline(x["Maturity"]), 
                                                      x["Maturity"], 
                                                      x["ImpliedVol"], 
                                                      qSpline(x["Maturity"]), 
                                                      rIntegralSpline(x["Maturity"]), 
                                                      qIntegralSpline(x["Maturity"]))
  
  res = np.reshape(np.array(list(zip(x_train.apply(impliedPriceFunction,axis=1).values))),(x_train.shape[0], 6))  # put greeks
  prices = res[:,0]
  deltas = res[:,1]
  vegas = res[:,2]
  delta_ks = res[:,3]
  gamma_ks = res[:,4]
  delta_Ts = res[:,5]
  
  sigmaRef = 0.25
  impliedPriceFunction = lambda x : generalizedGreeks(cp, 
                                                      S0, 
                                                      x["Strike"] , 
                                                      rSpline(x["Maturity"]), 
                                                      x["Maturity"], 
                                                      sigmaRef, 
                                                      qSpline(x["Maturity"]), 
                                                      rIntegralSpline(x["Maturity"]), 
                                                      qIntegralSpline(x["Maturity"]))
  
  res1 = np.reshape(np.array(list(zip(x_train.apply(impliedPriceFunction,axis=1).values))),(x_train.shape[0], 6))  # put greeks

  changedVar = changeOfVariable(x_train["Strike"],x_train["Maturity"])
  
  multiIndex = impliedVol.index
  
  cols = ["Price", "Delta", "Vega", "Delta Strike", "Gamma Strike", 
          "Theta", "ChangedStrike", "DividendFactor", "Strike", "Maturity", "ImpliedVol", "VegaRef"]

  dfData = np.vstack((prices, deltas, vegas, delta_ks, gamma_ks, delta_Ts) + 
                     changedVar + (x_train["Strike"], x_train["Maturity"], x_train["ImpliedVol"], res1[:,2]))
  df = pd.DataFrame(dfData.T , columns=cols, index = multiIndex)
  

  return df

In [0]:
testingDataSet = generateTestingData(formattedTestingData, S0, riskFreeIntegral, divSpreadIntegral, riskCurvespline, divSpline)
testingDataSet.tail()

In [0]:
#Checking call put parity
maturity = testingData.iloc[-4]["Maturity"]
strike = testingData.iloc[-4]["Strike"]
(S0 * np.exp(-divSpreadIntegral(maturity))  - np.exp(-riskFreeIntegral(maturity)) * strike) 

In [0]:
testingData.iloc[-4]["Option price"] - testingDataSet.iloc[-4]["Price"]

In [0]:
trainingDataSet = generateTestingData(formattedTrainingData["ImpliedVol"], S0, riskFreeIntegral, divSpreadIntegral, riskCurvespline, divSpline)
trainingDataSet.tail()

In [0]:
filteredTrainingData.tail()

In [0]:
filteredTrainingData[filteredTrainingData["Strike"]==5900]

In [0]:
trainingDataSet[trainingDataSet["Strike"]==5900]

In [0]:
filteredTrainingData[filteredTrainingData["Strike"]==4000]

In [0]:
trainingDataSet[trainingDataSet["Strike"]==4000]

In [0]:
localVolatility.head()

In [0]:
testingData.tail()

In [0]:
#Get local volatility from Crépey (2002) by nearest neighbour interpolation
def interpolatedLocalVolatility(localVol, priceGrid):
    strikeLocVol = np.ravel(localVol.index.get_level_values("Strike").values)
    maturityLocVol = np.ravel(localVol.index.get_level_values("Maturity").values)
    
    xym = np.vstack((strikeLocVol, maturityLocVol)).T
    opts = {'balanced_tree': False, 'compact_nodes': False}
    f =  interpolate.NearestNDInterpolator(xym,
                                           localVol["LocalVolatility"].values.flatten(), 
                                           rescale=True,
                                           tree_options=opts)

    strikePrice = priceGrid.index.get_level_values("Strike").values
    maturityPrice = priceGrid.index.get_level_values("Maturity").values
    func = lambda x : f(x[0],x[1])
    coordinates =  np.array( list( map( func, zip(strikePrice, maturityPrice) ) ) ).flatten() 

    return pd.Series(coordinates, index = priceGrid.index)

trainingDataSet["locvol"] = interpolatedLocalVolatility(localVolatility, trainingDataSet["Price"])
testingDataSet["locvol"] = interpolatedLocalVolatility(localVolatility, testingDataSet["Price"])

In [0]:
dataSet = trainingDataSet #Training set
dataSetTest = testingDataSet #Testing set

# Neural network 

## Scaling methods

Use min-max of scaling strike between 0 et 1 for improving stability of neural network training. 

In [0]:
def transformCustomMinMax(df, scaler):
  return pd.DataFrame(scaler.transform(df),
                      index = df.index, 
                      columns = df.columns)
#Reverse operation min-max scaling
def inverseTransformMinMax(df, scaler):
  return pd.DataFrame(scaler.inverse_transform(df),
                      index = df.index, 
                      columns = df.columns)
#Same thing but for a particular column
def inverseTransformColumnMinMax(originalDf, scaler, column):
  colIndex = originalDf.columns.get_loc(column.name)
  maxCol = scaler.data_max_[colIndex]
  minCol = scaler.data_min_[colIndex]
  return pd.Series(minCol + (maxCol - minCol) * column, index = column.index).rename(column.name)  
#Reverse transform of min-max scaling but for greeks   
def inverseTransformColumnGreeksMinMax(originalDf, 
                                       scaler,
                                       columnDerivative,
                                       columnFunctionName,
                                       columnVariableName,
                                       order = 1):
  colFunctionIndex = originalDf.columns.get_loc(columnFunctionName)
  maxColFunction = scaler.data_max_[colFunctionIndex]
  minColFunction = scaler.data_min_[colFunctionIndex]
  scaleFunction = (maxColFunction - minColFunction)
  
  colVariableIndex = originalDf.columns.get_loc(columnVariableName)
  maxColVariable = scaler.data_max_[colVariableIndex]
  minColVariable = scaler.data_min_[colVariableIndex]
  scaleVariable = (maxColVariable - minColVariable) ** order

  return pd.Series(scaleFunction * columnDerivative / scaleVariable , 
                   index = columnDerivative.index).rename(columnDerivative.name) 

In [0]:
#Tools functions for min-max scaling
def transformCustomId(df, scaler):
  return pd.DataFrame(df,
                      index = df.index, 
                      columns = df.columns)
def inverseTransformId(df, scaler):
  return pd.DataFrame(df,
                      index = df.index, 
                      columns = df.columns)
def inverseTransformColumnId(originalDf, scaler, column):
  return pd.Series(column, index = column.index).rename(column.name)  

def inverseTransformColumnGreeksId(originalDf, scaler, 
                                 columnDerivative, 
                                 columnFunctionName, 
                                 columnVariableName,
                                 order = 1):
  return pd.Series(columnDerivative , index = columnDerivative.index).rename(columnDerivative.name)


In [0]:
activateScaling = False
transformCustom = transformCustomMinMax if activateScaling else transformCustomId
inverseTransform = inverseTransformMinMax if activateScaling else inverseTransformId
inverseTransformColumn = inverseTransformColumnMinMax if activateScaling else inverseTransformColumnId
inverseTransformColumnGreeks = inverseTransformColumnGreeksMinMax if activateScaling else inverseTransformColumnGreeksId

In [0]:
scaler = skl.preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler.fit(dataSet)
scaledDataSet = transformCustom(dataSet, scaler)
scaledDataSetTest = transformCustom(dataSetTest, scaler)

In [0]:
scaledDataSet.head()

In [0]:
#Search strike for ATM option
midS0 = dataSet[dataSet.index.get_level_values("Strike") >= S0[0]].index.get_level_values("Strike").min()

## Plot functions

In [0]:
#Plot loss for each epoch 
def plotEpochLoss(lossSerie):
  fig = plt.figure(figsize=(20,10))
  ax = fig.gca()
  
  ax.plot(lossSerie , "-", color="black")
  ax.set_xlabel("Epoch number", fontsize=18, labelpad=20)
  ax.set_ylabel("Logarithmic Loss", fontsize=18, labelpad=20)
  ax.set_title("Training Loss evolution", fontsize=24)
  ax.tick_params(labelsize=16)
  ax.set_facecolor('white')
  plt.show()
  return

In [0]:
KMin = 0.7 * S0[0]
KMax = 1.3 * S0[0]


In [0]:
#Plot a surface as a superposition of curves
def plotMultipleCurve(data,
                      Title = 'True Price Surface',
                      yMin = KMin,
                      yMax = KMax,
                      zAsPercent = False):
  

  dataCurve = data[(data.index.get_level_values("Strike") <= yMax) * (data.index.get_level_values("Strike") >= yMin)]

  fig = plt.figure(figsize=(20,10))
  ax = fig.gca()

  for t in np.linspace(0,0.8,9) :
    k = dataCurve[dataCurve.index.get_level_values("Maturity") >= t].index.get_level_values("Maturity").unique().min()
    curveK = dataCurve[dataCurve.index.get_level_values("Maturity")==k]
    dataSerie = pd.Series(curveK.values * (100 if zAsPercent else 1) ,
                          index = curveK.index.get_level_values("Strike"))
    ax.plot(dataSerie , "--+", label=str(k))
  ax.legend()  
  ax.set_xlabel(data.index.names[0], fontsize=18, labelpad=20)
  ax.set_ylabel(data.name, fontsize=18, labelpad=20)
  if zAsPercent :
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
  ax.set_title(Title, fontsize=24)
  ax.tick_params(labelsize=16)
  ax.set_facecolor('white')
  plt.show()
  return

In [0]:
plotMultipleCurve(localVolatility["LocalVolatility"][localVolatility.index.get_level_values("Maturity")>0.01],
                  Title = 'Local Volatility Surface',
                  yMin=0.7*S0[0],
                  yMax=1.4*S0[0], 
                  zAsPercent=True)

In [0]:
#Plotting function for surface
#xTitle : title for x axis
#yTitle : title for y axis
#zTitle : title for z axis
#Title : plot title
#az : azimuth i.e. angle of view for surface
#yMin : minimum value for y axis
#yMax : maximum value for y axis
#zAsPercent : boolean, if true format zaxis as percentage 
def plotGridCustom(coordinates, zValue,
                   xTitle = "Maturity",
                   yTitle = "Strike",
                   zTitle = "Price",
                   Title = 'True Price Surface', 
                   az=320, 
                   yMin = KMin,
                   yMax = KMax,
                   zAsPercent = False):
  y = coordinates[:,0]
  filteredValue = (y > yMin) & (y < yMax)
  x = coordinates[:,1][filteredValue]
  y = coordinates[:,0][filteredValue]
  z = zValue[filteredValue].flatten()
  
  fig = plt.figure(figsize=(20,10))
  ax = fig.gca(projection='3d')
  
  ax.set_xlabel(xTitle, fontsize=18, labelpad=20)
  ax.set_ylabel(yTitle, fontsize=18, labelpad=20)
  ax.set_zlabel(zTitle, fontsize=18, labelpad=10)
  
  cmap=plt.get_cmap("inferno")
  colors=cmap(z * 100 if zAsPercent else z)[np.newaxis, :, :3]
  surf = ax.plot_trisurf(x, y,
                         z * 100 if zAsPercent else z ,
                         linewidth=1.0,
                         antialiased=True, 
                         cmap = cmap,
                         color=(0,0,0,0))
  scaleEdgeValue = surf.to_rgba(surf.get_array())
  surf.set_edgecolors(scaleEdgeValue) 
  surf.set_alpha(0)

  if zAsPercent :
    ax.zaxis.set_major_formatter(mtick.PercentFormatter())
  ax.view_init(elev=10., azim=az)
  ax.set_title(Title, fontsize=24)
  ax.set_facecolor('white')

  plt.tick_params(labelsize=16)

  
  plt.show()


  return

In [0]:
#Plotting function from a dataframe
def plotSurface(data, 
                zName, 
                Title = 'True Price Surface', 
                az=320,
                yMin = KMin,
                yMax = KMax,
                zAsPercent = False):
  plotGridCustom(dataSet.index.to_frame().values, 
                 data[zName].values,
                 xTitle = dataSet.index.names[1],
                 yTitle = dataSet.index.names[0],
                 zTitle = zName,
                 Title = Title, 
                 az=az, 
                 yMin = yMin, 
                 yMax = yMax, 
                 zAsPercent=zAsPercent)
  return

#Plotting function from a pandas series
def plotSerie(data,
              Title = 'True Price Surface',
              az=320,
              yMin = KMin,
              yMax = KMax, 
              zAsPercent = False):
  

  plotGridCustom(data.index.to_frame().values, 
                 data.values,
                 xTitle = dataSet.index.names[1],
                 yTitle = dataSet.index.names[0],
                 zTitle = data.name,
                 Title = Title, 
                 az=az, 
                 yMin = yMin, 
                 yMax = yMax, 
                 zAsPercent = zAsPercent)
  return

In [0]:
plt.get_cmap("plasma")(0)

In [0]:
plotSurface(dataSet, "Price", Title = 'True Price Surface')

In [0]:
inverseTransform(scaledDataSet, scaler).head()

In [0]:
#Plot predicted value, benchmark value, absoluate error and relative error
#It also compute RMSE between predValue and refValue
#predValue : approximated value 
#refValue : benchamrk value
#quantityName : name for approximated quantity
#az : azimuth i.e. angle of view for surface
#yMin : minimum value for y axis
#yMax : maximum value for y axis
def predictionDiagnosis(predValue, 
                        refValue, 
                        quantityName, 
                        az=320,
                        yMin = KMin,
                        yMax = KMax):
  title = "Predicted " + quantityName + " surface"
  plotSerie(predValue.rename(quantityName), 
            Title = title, 
            az=az,
            yMin = yMin,
            yMax = yMax)
  
  title = "True " + quantityName + " surface"
  plotSerie(refValue.rename(quantityName), 
            Title = title, 
            az=az,
            yMin = yMin,
            yMax = yMax)
  
  title = quantityName + " surface error"
  absoluteError = np.abs(predValue - refValue) 
  plotSerie(absoluteError.rename(quantityName + " Absolute Error"),
            Title = title,
            az=az,
            yMin = yMin,
            yMax = yMax)
  
  title = quantityName + " surface error"
  relativeError = np.abs(predValue - refValue) / refValue
  plotSerie(relativeError.rename(quantityName + " Relative Error (%)"),
            Title = title,
            az=az,
            yMin = yMin,
            yMax = yMax, 
            zAsPercent = True)
  
  print("RMSE : ", np.sqrt(np.mean(np.square(absoluteError))) )
  
  return

#Diagnose Price, theta, gamma and local volatility
def modelSummary(price, 
                 volLocale, 
                 delta_T, 
                 gamma_K, 
                 benchDataset,
                 sigma=0.3, 
                 az=40,
                 yMin = KMin,
                 yMax = KMax):
  priceRef = benchDataset["Price"]
  predictionDiagnosis(price, 
                      priceRef, 
                      "Price",
                      az=320,
                      yMin = yMin,
                      yMax = yMax)
  
  title = "Price" + " scaled surface error"
  absoluteError = np.abs(price - priceRef) / benchDataset["VegaRef"]
  plotSerie(absoluteError.rename("Price" + " Absolute Error normalized by vega 30%"),
            Title = title,
            az=320,
            yMin = yMin,
            yMax = yMax)
  
  volLocaleRef = benchDataset["locvol"]
  predictionDiagnosis(volLocale, 
                      volLocaleRef, 
                      "Local volatility",
                      az=az,
                      yMin = yMin,
                      yMax = yMax)
  
  dTRef = benchDataset["Theta"]
  predictionDiagnosis(delta_T, 
                      dTRef, 
                      "Theta",
                      az=340,
                      yMin = yMin,
                      yMax = yMax)
  
  gKRef = benchDataset["Gamma Strike"]
  predictionDiagnosis(gamma_K, 
                      gKRef, 
                      "Gamma Strike",
                      az=340,
                      yMin = yMin,
                      yMax = yMax)
  return
  

### Implied volatility function calibration by bissection

In [0]:
def bs_price(cp, s, k, rf, t, v, div):
        """ Price an option using the Black-Scholes model.
        cp: +1/-1 for call/put
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        """
    
        d1 = (np.log(s/k)+(rf-div+0.5*v*v)*t)/(v*np.sqrt(t))
        d2 = d1 - v*np.sqrt(t)

        optprice = (cp*s*np.exp(-div*t)*st.norm.cdf(cp*d1)) - (cp*k*np.exp(-rf*t)*st.norm.cdf(cp*d2))
        
        return optprice

def bissectionMethod(S_0, r, q, implied_vol0, maturity, Strike, refPrice, epsilon):
    calibratedSigma = implied_vol0
    #Call black-scholes price function for initial value
    priceBS = bs_price(-1 ,S0, Strike, r, maturity, calibratedSigma, q)
    sigmaUp = 2.0
    sigmaInf = epsilon
    lossSerie = []
    
    priceMax = bs_price(-1 ,S0, Strike, r, maturity, sigmaUp, q)
    if priceMax < refPrice:
        return priceMax, sigmaUp, pd.Series(lossSerie)
    
    priceMin = bs_price(-1 ,S0, Strike, r, maturity, sigmaInf, q)
    if priceMin > refPrice:
        return priceMin, sigmaInf, pd.Series(lossSerie) 

    #Stop the optimization when the error is less than epsilon
    while(abs(priceBS - refPrice) > epsilon):
        #Update the upper bound or the lower bound 
        #by comparing calibrated price and the target price 
        if priceBS < refPrice : 
            sigmaInf = calibratedSigma
        else :
            sigmaUp = calibratedSigma
        #Update calibratedSigma
        calibratedSigma = (sigmaUp + sigmaInf) / 2
        #Update calibrated price
        priceBS = bs_price(-1 ,S0, Strike, r, maturity, calibratedSigma, q)
        #Record the calibration error for this step
        lossSerie.append(abs(priceBS - refPrice)) 
        
    return priceBS, calibratedSigma, pd.Series(lossSerie)

In [0]:
#Execute calibration of implied volatility from estimated price and benchmark price
#Then plot esitmated implied vol, absolute and relative error
def plotImpliedVol(priceSurface, 
                   refImpliedVol, 
                   rIntegralSpline = None, 
                   qIntegralSpline = None, 
                   az=40,
                   yMin = KMin,
                   yMax = KMax,
                   relativeErrorVolMax = 10):
    df = priceSurface.index.to_frame()
    df["Price"] = priceSurface

    epsilon = 1e-9
    calibrationFunction = lambda x : bissectionMethod(S0, 
                                                      rIntegralSpline(x["Maturity"])/x["Maturity"] if (rIntegralSpline is not None) else r, 
                                                      qIntegralSpline(x["Maturity"])/x["Maturity"] if (qIntegralSpline is not None) else q, 
                                                      0.2, 
                                                      x["Maturity"], 
                                                      x["Strike"], 
                                                      x["Price"], 
                                                      epsilon)[1]

    impliedVol = df.apply(calibrationFunction, axis = 1).rename("Implied Volatility")
    impliedVolError = np.abs(impliedVol-refImpliedVol).rename('Absolute Error')
    relativeImpliedVolError = (impliedVolError / refImpliedVol).rename("Relative error (%)")
    
    plotSerie(impliedVol, 
              Title = 'Implied volatility surface', 
              az=az,
              yMin = yMin,
              yMax = yMax)

    plotSerie(impliedVolError, 
              Title = 'Implied volatility error', 
              az=az,
              yMin = yMin,
              yMax = yMax)
    
    plotSerie(relativeImpliedVolError.clip(0,relativeErrorVolMax / 100.0), 
              Title = 'Implied volatility relative error', 
              az=az,
              yMin = yMin,
              yMax = yMax,
              zAsPercent = True)
  
    print("Implied volalitity RMSE : ", np.sqrt(np.mean(np.square(impliedVolError))) )

    return impliedVol

In [0]:
%matplotlib inline

In [0]:
plotSerie(localVolatility["LocalVolatility"][localVolatility.index.get_level_values("Maturity")>0.01],
          Title = 'Local Volatility Surface',
          az=30,
          yMin=0.7*S0,
          yMax=1.4*S0, zAsPercent=True)

## Learning Price

In [0]:
#Import tensorflow for 1.x version 
from keras.layers import Dense, Input
from keras import Model
import keras.backend as K
import keras.activations as Act
from functools import partial
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
#Deactivate warning messages
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [0]:
hyperparameters = {}
#penalization coefficient
hyperparameters["lambdaLocVol"] = 100
hyperparameters["lambdaSoft"] = 100 
hyperparameters["lambdaGamma"] = 10000

#Derivative soft constraints parameters
hyperparameters["lowerBoundTheta"] = 0.01
hyperparameters["lowerBoundGamma"] = 0.00001

#Local variance parameters
hyperparameters["DupireVarCap"] = 10
hyperparameters["DupireVolLowerBound"] = 0.05
hyperparameters["DupireVolUpperBound"] = 0.40

#Learning scheduler coefficient
hyperparameters["LearningRateStart"] = 0.1
hyperparameters["Patience"] = 100
hyperparameters["batchSize"] = 50
hyperparameters["FinalLearningRate"] = 1e-6
hyperparameters["FixedLearningRate"] = False

#Training parameters
hyperparameters["nbUnits"] = 200 #number of units for hidden layers
hyperparameters["maxEpoch"] = 10000 #maximum number of epochs

### Learning scheduler

In [0]:
#Format result from training step
def evalAndFormatResult(price, loss, dataSet):

    scaledPredPrice = pd.Series(price.flatten(), index = dataSet.index).rename("Price")
    predPrice = inverseTransformColumn(dataSet, scaler, scaledPredPrice)
    
    return predPrice, pd.Series(loss)

#Format result from training step when local volatility is computed
def evalAndFormatDupireResult(price, volDupire, theta, gamma, dupireVar, loss, dataSet):
    predPrice, lossEpoch = evalAndFormatResult(price, loss, dataSet)

    predDupire = pd.Series(volDupire.flatten(), index = dataSet.index).rename("Dupire")
    
    scaledTheta = pd.Series(theta.flatten(), index = dataSet.index).rename("Theta")
    predTheta = inverseTransformColumnGreeks(dataSet, scaler, scaledTheta, 
                                             "Price", "Maturity")
    
    scaledGammaK = pd.Series(gamma.flatten(), index = dataSet.index).rename("GammaK")
    predGammaK = inverseTransformColumnGreeks(dataSet, scaler, scaledGammaK, 
                                              "Price", "ChangedStrike", order = 2)
    
    return predPrice, predDupire, predTheta, predGammaK, lossEpoch



In [0]:
#Penalization for pseudo local volatility
def intervalRegularization(localVariance, vegaRef, hyperParameters):
  lowerVolBound = hyperParameters["DupireVolLowerBound"]
  upperVolBound = hyperParameters["DupireVolUpperBound"]
  no_nans = tf.clip_by_value(localVariance, 0, hyperParameters["DupireVarCap"])
  reg = tf.nn.relu(tf.square(lowerVolBound) - no_nans) + tf.nn.relu(no_nans - tf.square(upperVolBound))
  lambdas = hyperParameters["lambdaLocVol"] / tf.reduce_mean(vegaRef)
  return lambdas * tf.reduce_mean(tf.boolean_mask(reg, tf.is_finite(reg)))

#Add above regularization to the list of penalization
def addDupireRegularisation(priceTensor, tensorList, penalizationList, formattingResultFunction, vegaRef, hyperParameters):
    updatedPenalizationList = penalizationList + [intervalRegularization(tensorList[-1], vegaRef, hyperParameters)]
    return priceTensor, tensorList, updatedPenalizationList, formattingResultFunction

In [0]:
#Mini-batch sampling methods for large datasets
def selectMiniBatchWithoutReplacement(dataSet, batch_size):
    nbObs = dataSet.shape[0]
    idx = np.arange(nbObs) 
    np.random.shuffle(idx) 
    nbBatches = int(np.ceil(nbObs/batch_size))
    xBatchList = []
    lastBatchIndex = 0
    for i in range(nbBatches):
        firstBatchIndex = i*batch_size
        lastBatchIndex = (i+1)*batch_size
        xBatchList.append(dataSet.iloc[idx[firstBatchIndex:lastBatchIndex],:])
    xBatchList.append(dataSet.iloc[idx[lastBatchIndex:],:])
    return xBatchList

def selectMiniBatchWithReplacement(dataSet, batch_size):
    nbObs = dataSet.shape[0] 
    nbBatches = int(np.ceil(nbObs/batch_size)) + 1
    xBatchList = []
    lastBatchIndex = 0
    for i in range(nbBatches):
        idx = np.random.randint(nbObs, size = batch_size)
        xBatchList.append(dataSet.iloc[idx,:])
    return xBatchList


In [0]:
#Train neural network with a decreasing rule for learning rate
#NNFactory :  function creating the architecture
#dataSet : training data
#activateRegularization : boolean, if true add bound penalization to dupire variance
#hyperparameters : dictionnary containing various hyperparameters
#modelName : name under which tensorflow model is saved
def create_train_model(NNFactory, 
                       dataSet, 
                       activateRegularization, 
                       hyperparameters,
                       modelName = "bestModel"):
    hidden_nodes = hyperparameters["nbUnits"] 
    nbEpoch = hyperparameters["maxEpoch"] 
    fixedLearningRate = (None if hyperparameters["FixedLearningRate"] else hyperparameters["LearningRateStart"])
    patience = hyperparameters["Patience"]
    
    # Go through num_iters iterations (ignoring mini-batching)
    activateLearningDecrease = (~ hyperparameters["FixedLearningRate"])
    learningRate = hyperparameters["LearningRateStart"]
    learningRateEpoch = 0
    finalLearningRate = hyperparameters["FinalLearningRate"]

    batch_size = hyperparameters["batchSize"]

    start = time.time()
    # Reset the graph
    tf.reset_default_graph()
    
    # Placeholders for input and output data   
    Strike = tf.placeholder(tf.float32,[None,1])
    Maturity = tf.placeholder(tf.float32,[None,1])
    factorPrice = tf.placeholder(tf.float32,[None,1])
    y = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y')
    vegaRef = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='vegaRef')
    learningRateTensor = tf.placeholder(tf.float32,[])
    
    #Get scaling for strike
    colStrikeIndex = dataSet.columns.get_loc("ChangedStrike")
    maxColFunction = scaler.data_max_[colStrikeIndex]
    minColFunction = scaler.data_min_[colStrikeIndex]
    scF = (maxColFunction - minColFunction) 
    scaleTensor = tf.constant(scF, dtype=tf.float32)
    strikeMinTensor = tf.constant(minColFunction, dtype=tf.float32)

    price_pred_tensor = None
    TensorList = None
    penalizationList = None 
    formattingFunction = None
    if activateRegularization : #Add pseudo local volatility regularisation
        price_pred_tensor, TensorList, penalizationList, formattingFunction = addDupireRegularisation( *NNFactory(hidden_nodes,
                                                                                                                  Strike,
                                                                                                                  Maturity, 
                                                                                                                  scaleTensor, 
                                                                                                                  strikeMinTensor, 
                                                                                                                  vegaRef, 
                                                                                                                  hyperparameters) ,
                                                                                                      vegaRef, 
                                                                                                      hyperparameters)
    else :
        price_pred_tensor, TensorList, penalizationList, formattingFunction = NNFactory(hidden_nodes,
                                                                                        Strike, 
                                                                                        Maturity, 
                                                                                        scaleTensor, 
                                                                                        strikeMinTensor, 
                                                                                        vegaRef, 
                                                                                        hyperparameters)

    price_pred_tensor_sc= tf.multiply( factorPrice, price_pred_tensor)
    TensorList[0] = price_pred_tensor_sc
    
    # Define a loss function
    pointwiseError = tf.reduce_mean(tf.abs(price_pred_tensor_sc - y) / vegaRef)
    errors = tf.add_n([pointwiseError] + penalizationList) 
    loss = tf.log(tf.reduce_mean(errors))



    # Define a train operation to minimize the loss
    lr = learningRate

    optimizer = tf.train.AdamOptimizer(learning_rate=learningRateTensor)
    train = optimizer.minimize(loss)

    # Initialize variables and run session
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(init)
    n = dataSet.shape[0]
    scaledInput = transformCustomMinMax(dataSet, scaler)

    
    maturity = dataSet["Maturity"].values.reshape(n,1)
    loss_serie = []

    def createFeedDict(batch):
        batchSize = batch.shape[0]
        feedDict = {Strike : scaledInput["ChangedStrike"].loc[batch.index].values.reshape(batchSize,1),
                    Maturity : batch["Maturity"].values.reshape(batchSize,1), 
                    y : batch["Price"].values.reshape(batchSize,1),
                    factorPrice : batch["DividendFactor"].values.reshape(batchSize,1), 
                    learningRateTensor : learningRate,
                    vegaRef : np.ones_like(batch["VegaRef"].values.reshape(batchSize,1))}
        return feedDict

    #Learning rate is divided by 10 if no imporvement is observed for training loss after "patience" epochs
    def updateLearningRate(iterNumber, lr, lrEpoch):
        if not activateLearningDecrease :
            print("Constant learning rate, stop training")
            return False, lr, lrEpoch
        if learningRate > finalLearningRate :
            lr *= 0.1
            lrEpoch = iterNumber
            saver.restore(sess, modelName)
            print("Iteration : ", lrEpoch, "new learning rate : ", lr)
        else :
          print("Last Iteration : ", lrEpoch, "final learning rate : ", lr)
          return False, lr, lrEpoch
        return True, lr, lrEpoch
    
    epochFeedDict = createFeedDict(dataSet)

    def evalBestModel():
        if not activateLearningDecrease :
            print("Learning rate : ", learningRate, " final loss : ", min(loss_serie))
        currentBestLoss = sess.run(loss, feed_dict=epochFeedDict)
        currentBestPenalizations = sess.run([pointwiseError, penalizationList], feed_dict=epochFeedDict)
        print("Best loss (hidden nodes: %d, iterations: %d): %.2f" % (hidden_nodes, len(loss_serie), currentBestLoss))
        print("Best Penalization : ", currentBestPenalizations)
        return
    
    for i in range(nbEpoch):
        miniBatchList = [dataSet]
        penalizationResult = sess.run(penalizationList, feed_dict=epochFeedDict)
        lossResult = sess.run(pointwiseError, feed_dict=epochFeedDict)

        #miniBatchList = selectMiniBatchWithoutReplacement(dataSet, batch_size)
        for k in range(len(miniBatchList)) :
            batchFeedDict = createFeedDict(miniBatchList[k])
            sess.run(train, feed_dict=batchFeedDict)
        
        
        loss_serie.append(sess.run(loss, feed_dict=epochFeedDict))

        if (len(loss_serie) < 2) or (loss_serie[-1] <= min(loss_serie)):
          #Save model as model is improved
          saver.save(sess, modelName)
        if (np.isnan(loss_serie[-1]) or  #Unstable model
            ( (i-learningRateEpoch >= patience) and (min(loss_serie[-patience:]) > min(loss_serie)) ) ) : #No improvement for training loss during the latest 100 iterations
          continueTraining, learningRate, learningRateEpoch = updateLearningRate(i, learningRate, learningRateEpoch)
          if continueTraining :
            evalBestModel()
          else :
            break
    saver.restore(sess, modelName)  
    
    evalBestModel()

    evalList  = sess.run(TensorList, feed_dict=epochFeedDict)
    
    sess.close()
    end = time.time()
    print("Training Time : ", end - start)
    
    return formattingFunction(*evalList, loss_serie, dataSet) 

In [0]:
#Evaluate neural network without training, it restores parameters obtained from a pretrained model 
#NNFactory :  function creating the neural architecture
#dataSet : dataset on which neural network is evaluated 
#activateRegularization : boolean, if true add bound penalization for dupire variance
#hyperparameters : dictionnary containing various hyperparameters
#modelName : name of tensorflow model to restore
def create_eval_model(NNFactory, 
                      dataSet, 
                      activateRegularization, 
                      hyperparameters,
                      modelName = "bestModel"):
    hidden_nodes = hyperparameters["nbUnits"] 
    
    # Go through num_iters iterations (ignoring mini-batching)
    activateLearningDecrease = (~ hyperparameters["FixedLearningRate"])
    learningRate = hyperparameters["LearningRateStart"]

    # Reset the graph
    tf.reset_default_graph()
    
    # Placeholders for input and output data   
    Strike = tf.placeholder(tf.float32,[None,1])
    Maturity = tf.placeholder(tf.float32,[None,1])
    factorPrice = tf.placeholder(tf.float32,[None,1])
    y = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y')
    vegaRef = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='vegaRef')
    learningRateTensor = tf.placeholder(tf.float32,[])
    
    #Get scaling for strike
    colStrikeIndex = dataSet.columns.get_loc("ChangedStrike")
    maxColFunction = scaler.data_max_[colStrikeIndex]
    minColFunction = scaler.data_min_[colStrikeIndex]
    scF = (maxColFunction - minColFunction) 
    scaleTensor = tf.constant(scF, dtype=tf.float32)
    strikeMinTensor = tf.constant(minColFunction, dtype=tf.float32)

    price_pred_tensor = None
    TensorList = None
    penalizationList = None 
    formattingFunction = None
    if activateRegularization : 
        price_pred_tensor, TensorList, penalizationList, formattingFunction = addDupireRegularisation( *NNFactory(hidden_nodes,
                                                                                                                  Strike,
                                                                                                                  Maturity, 
                                                                                                                  scaleTensor, 
                                                                                                                  strikeMinTensor, 
                                                                                                                  vegaRef,
                                                                                                                  hyperparameters,
                                                                                                                  IsTraining=False), 
                                                                                                      vegaRef,
                                                                                                      hyperparameters )
    else :
        price_pred_tensor, TensorList, penalizationList, formattingFunction = NNFactory(hidden_nodes,
                                                                                        Strike, 
                                                                                        Maturity, 
                                                                                        scaleTensor, 
                                                                                        strikeMinTensor,
                                                                                        vegaRef,
                                                                                        hyperparameters,
                                                                                        IsTraining=False)

    price_pred_tensor_sc= tf.multiply(factorPrice,price_pred_tensor)
    TensorList[0] = price_pred_tensor_sc
    
    # Define a loss function
    pointwiseError = tf.reduce_mean(tf.abs(price_pred_tensor_sc - y) / vegaRef)
    errors = tf.add_n([pointwiseError] + penalizationList)
    loss = tf.log(tf.reduce_mean(errors))


    # Define a train operation to minimize the loss
    lr = learningRate 

    optimizer = tf.train.AdamOptimizer(learning_rate=learningRateTensor)
    train = optimizer.minimize(loss)

    # Initialize variables and run session
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(init)
    n = dataSet.shape[0]
    scaledInput = transformCustomMinMax(dataSet, scaler)

    
    maturity = dataSet["Maturity"].values.reshape(n,1)
    loss_serie = []

    def createFeedDict(batch):
        batchSize = batch.shape[0]
        feedDict = {Strike : scaledInput["ChangedStrike"].loc[batch.index].values.reshape(batchSize,1),
                    Maturity : batch["Maturity"].values.reshape(batchSize,1), 
                    y : batch["Price"].values.reshape(batchSize,1),
                    factorPrice : batch["DividendFactor"].values.reshape(batchSize,1), 
                    learningRateTensor : learningRate,
                    vegaRef : np.ones_like(batch["VegaRef"].values.reshape(batchSize,1))}
        return feedDict
    
    epochFeedDict = createFeedDict(dataSet)

    def evalBestModel():
        if not activateLearningDecrease :
            print("Learning rate : ", learningRate, " final loss : ", min(loss_serie))
        currentBestLoss = sess.run(loss, feed_dict=epochFeedDict)
        currentBestPenalizations = sess.run([pointwiseError, penalizationList], feed_dict=epochFeedDict)
        print("Best loss (hidden nodes: %d, iterations: %d): %.2f" % (hidden_nodes, len(loss_serie), currentBestLoss))
        print("Best Penalization : ", currentBestPenalizations)
        return
    
    saver.restore(sess, modelName)  
    
    evalBestModel()

    evalList  = sess.run(TensorList, feed_dict=epochFeedDict)
    
    sess.close()
    
    return formattingFunction(*evalList, [0], dataSet)

### Convex architecture (Price only)

In [0]:

#Soft constraints for strike convexity and strike/maturity monotonicity  
def arbitragePenalties(priceTensor, strikeTensor, maturityTensor, scaleTensor, vegaRef, hyperparameters):

    dK = tf.gradients(priceTensor, strikeTensor, name="dK")
    hK = tf.gradients(dK[0], strikeTensor, name="hK") / tf.square(scaleTensor)
    theta = tf.gradients(priceTensor,maturityTensor,name="dT")
    
    lambdas = hyperparameters["lambdaSoft"]  / tf.reduce_mean(vegaRef) 
    lowerBoundTheta = tf.constant(hyperparameters["lowerBoundTheta"])
    lowerBoundGamma = tf.constant(hyperparameters["lowerBoundGamma"])
    grad_penalty = lambdas * tf.reduce_mean(tf.nn.relu(-theta[0] + lowerBoundTheta ))
    hessian_penalty = lambdas * hyperparameters["lowerBoundGamma"] * tf.reduce_mean(tf.nn.relu(-hK[0] + lowerBoundGamma ))
    
    return [grad_penalty, hessian_penalty]

In [0]:
#Tools function for Neural network architecture

#Initilize weights as positive
def positiveKernelInitializer(shape, 
                              dtype=None, 
                              partition_info=None):
  return tf.abs(tf.keras.initializers.normal()(shape,dtype=dtype, partition_info=partition_info))

#Soft convex layer
def convexLayer(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    return tf.nn.softplus(layer)

#Soft monotonic layer
def monotonicLayer(n_units,  tensor, isTraining, name):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor, 
                            units=n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    
    return tf.nn.sigmoid(layer)

#Soft convex layer followed by output layer for regression 
def convexOutputLayer(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=2*n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal(),
                            activation = 'softplus')
    
     
    layer = tf.layers.dense(layer, 
                            units=1,
                            kernel_initializer=positiveKernelInitializer,
                            activation = 'softplus')
    
    return layer




#Neural network factory for Hybrid approach : splitted network with soft contraints
def NNArchitectureConstrained(n_units, 
                              strikeTensor,
                              maturityTensor, 
                              scaleTensor, 
                              strikeMinTensor, 
                              vegaRef, 
                              hyperparameters,
                              IsTraining=True):
  #First splitted layer
  hidden1S = convexLayer(n_units = n_units,
                         tensor = strikeTensor,
                         isTraining=IsTraining, 
                         name = "Hidden1S")
  
  hidden1M = monotonicLayer(n_units = n_units,
                            tensor = maturityTensor, 
                            isTraining = IsTraining, 
                            name = "Hidden1M")
  
  hidden1 = tf.concat([hidden1S, hidden1M], axis=-1)
  
  #Second and output layer
  out = convexOutputLayer(n_units = n_units,
                          tensor = hidden1,
                          isTraining = IsTraining,
                          name = "Output")
  #Soft constraints
  penaltyList = arbitragePenalties(out, strikeTensor, 
                                   maturityTensor, 
                                   scaleTensor, 
                                   vegaRef, 
                                   hyperparameters)
  
  return out, [out], penaltyList, evalAndFormatResult

In [0]:
plt.plot(dataSet.index.get_level_values("Strike"), dataSet["Price"])

In [0]:
y_pred0, lossSerie0 = create_train_model(NNArchitectureConstrained, scaledDataSet, False, hyperparameters, modelName = "softConvexHybridModel")

In [0]:
print("Minimum error : ",lossSerie0.min())
plotEpochLoss(lossSerie0)

In [0]:
lossSerie0.iloc[-1]

In [0]:
y_pred0, lossSerie0 = create_eval_model(NNArchitectureConstrained, 
                                        scaledDataSet, 
                                        False, 
                                        hyperparameters, 
                                        modelName = "softConvexHybridModel")
predictionDiagnosis(y_pred0, dataSet["Price"], " Price ")
impV0 = plotImpliedVol(y_pred0, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred0.head()

In [0]:
y_pred0.loc[(midS0,slice(None))].head()

In [0]:
dataSet["Price"].head()

In [0]:
y_pred0Test, lossSerie0Test = create_eval_model(NNArchitectureConstrained, 
                                                scaledDataSetTest, 
                                                False, 
                                                hyperparameters, 
                                                modelName = "softConvexHybridModel")
predictionDiagnosis(y_pred0Test, dataSetTest["Price"], " Price ")
impV0Test = plotImpliedVol(y_pred0Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Unconstrained neural network (Price only)

In [0]:
#Unconstrained dense layer
def unconstrainedLayer(n_units,  tensor, isTraining, name, activation = K.softplus):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor, 
                            units=n_units,
                            activation=activation,  
                            kernel_initializer=tf.keras.initializers.he_normal())
    return layer

#Factory for unconstrained network
def NNArchitectureUnconstrained(n_units, 
                                strikeTensor,
                                maturityTensor, 
                                scaleTensor, 
                                strikeMinTensor, 
                                vegaRef,
                                hyperparameters,
                                IsTraining=True):
  
  inputLayer = tf.concat([strikeTensor,maturityTensor], axis=-1)
  
  #First layer
  hidden1 = unconstrainedLayer(n_units = n_units,
                               tensor = inputLayer,
                               isTraining=IsTraining, 
                               name = "Hidden1")
  
  #Second layer
  hidden2 = unconstrainedLayer(n_units = n_units,
                               tensor = hidden1,
                               isTraining=IsTraining, 
                               name = "Hidden2")
  #Output layer 
  out = unconstrainedLayer(n_units = 1,
                           tensor = hidden2,
                           isTraining=IsTraining, 
                           name = "Output",
                           activation = None)
  
  return out, [out], [], evalAndFormatResult


In [0]:
y_pred1, lossSerie1 = create_train_model(NNArchitectureUnconstrained, 
                                         scaledDataSet, 
                                         False, 
                                         hyperparameters,
                                         modelName = "unconstrainedModel")

In [0]:
print("Minimum error : ",lossSerie1.min())
plotEpochLoss(lossSerie1)

In [0]:
lossSerie1.iloc[-1]

In [0]:
y_pred1, lossSerie1 = create_eval_model(NNArchitectureUnconstrained, 
                                        scaledDataSet, 
                                        False, 
                                        hyperparameters,
                                        modelName = "unconstrainedModel")
predictionDiagnosis(y_pred1, dataSet["Price"], " Price ")
impV1 = plotImpliedVol(y_pred1, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred1.head()

In [0]:
y_pred1.loc[(midS0,slice(None))].head()

In [0]:
dataSet["Price"].head()

In [0]:
y_pred1Test, lossSerie1Test = create_eval_model(NNArchitectureUnconstrained, 
                                                scaledDataSetTest, 
                                                False, 
                                                hyperparameters,
                                                modelName = "unconstrainedModel")
predictionDiagnosis(y_pred1Test, dataSetTest["Price"], " Price ")
impV1Test = plotImpliedVol(y_pred1Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

## Dupire formula implementation

In [0]:
#Dupire formula from exact derivative computation
def dupireFormula(HessianStrike, 
                  GradMaturity, 
                  Strike,
                  scaleTensor,
                  strikeMinTensor,
                  IsTraining=True):
  twoConstant = tf.constant(2.0)
  dupireVar = tf.math.divide(tf.math.divide(tf.math.scalar_mul(twoConstant,GradMaturity), 
                                            HessianStrike), 
                             tf.square(Strike + strikeMinTensor / scaleTensor))
  #Initial weights of neural network can be random which lead to negative dupireVar
  dupireVolTensor = tf.sqrt(dupireVar) 
  return dupireVolTensor, dupireVar

In [0]:
#Dupire formula with derivative obtained from native tensorflow algorithmic differentiation
def rawDupireFormula(priceTensor, 
                     adjustedStrikeTensor, 
                     maturityTensor,
                     scaleTensor,
                     strikeMinTensor,
                     IsTraining=True):
  batchSize = tf.shape(adjustedStrikeTensor)[0]
  dK = tf.reshape(tf.gradients(priceTensor, adjustedStrikeTensor, name="dK")[0], shape=[batchSize,-1])
  hK = tf.reshape(tf.gradients(dK, adjustedStrikeTensor, name="hK")[0], shape=[batchSize,-1])
  dupireDenominator = tf.square(adjustedStrikeTensor + strikeMinTensor / scaleTensor) * hK

  dT = tf.reshape(tf.gradients(priceTensor,maturityTensor,name="dT")[0], shape=[batchSize,-1])

  #Initial weights of neural network can be random which lead to negative dupireVar
  dupireVar = 2 * dT / dupireDenominator
  dupireVol = tf.sqrt(dupireVar) 
  return  dupireVol, dT, hK / tf.square(scaleTensor), dupireVar

### Hybrid architecture (Exact derivatives)

In [0]:
def exact_derivatives(Strike, Maturity):
    w1K = tf.get_default_graph().get_tensor_by_name( 'dense/kernel:0')
    w1T = tf.get_default_graph().get_tensor_by_name( 'dense_1/kernel:0')
    w2 = tf.get_default_graph().get_tensor_by_name( 'dense_2/kernel:0')
    w3 = tf.get_default_graph().get_tensor_by_name( 'dense_3/kernel:0')

    b1K = tf.get_default_graph().get_tensor_by_name( 'dense/bias:0')
    b1T = tf.get_default_graph().get_tensor_by_name( 'dense_1/bias:0')
    b2 = tf.get_default_graph().get_tensor_by_name( 'dense_2/bias:0')
    b3 = tf.get_default_graph().get_tensor_by_name( 'dense_3/bias:0')

    Z1K= tf.nn.softplus(tf.matmul(Strike, w1K) + b1K)
    Z1T= tf.nn.sigmoid(tf.matmul(Maturity, w1T) + b1T)

    Z= tf.concat([Z1K, Z1T], axis=-1)
    I2=tf.matmul(Z, w2) + b2
    Z2=tf.nn.softplus(I2)
    I3=tf.matmul(Z2, w3) + b3
    F=tf.nn.softplus(I3)

    D1K= tf.nn.sigmoid(tf.matmul(Strike, w1K) + b1K)
    I2K=tf.multiply(D1K, w1K)
    Z2K = tf.concat([I2K, tf.scalar_mul(tf.constant(0.0),I2K)],axis=-1)
   
    dI2dK=tf.matmul(Z2K, w2)
    Z2w3=tf.multiply(tf.nn.sigmoid(I2),dI2dK)
    dI3dK=tf.matmul(Z2w3, w3)
    dF_dK=tf.multiply(tf.nn.sigmoid(I3),dI3dK)
    
    D1T= sigmoidGradient(tf.matmul(Maturity,w1T) + b1T)
    I2T=tf.multiply(D1T, w1T)
    Z2T = tf.concat([tf.scalar_mul(tf.constant(0.0),I2T), I2T],axis=-1)
   
    dI2dT=tf.matmul(Z2T, w2)
    Z2w3=tf.multiply(tf.nn.sigmoid(I2),dI2dT)
    dI3dT=tf.matmul(Z2w3, w3)
    dF_dT=tf.multiply(tf.nn.sigmoid(I3),dI3dT)
    
    
    d2F_dK2=tf.multiply(sigmoidGradient(I3),tf.square(dI3dK))
    DD1K=sigmoidGradient(tf.matmul(Strike, w1K) + b1K)
    w1K2=tf.multiply(w1K,w1K)
    ID2K=tf.multiply(DD1K,w1K2)
    ZD2K = tf.concat([ID2K, tf.scalar_mul(tf.constant(0.0),ID2K)],axis=-1)
   
    d2I2_dK2=tf.matmul(ZD2K, w2)
    
    ZD2=tf.multiply(sigmoidGradient(I2), tf.square(dI2dK)) 
    ZD2+=tf.multiply(tf.nn.sigmoid(I2),d2I2_dK2)
    d2I3dK2=tf.matmul(ZD2, w3)
    
    d2F_dK2+=tf.multiply(tf.nn.sigmoid(I3),d2I3dK2)
    
    return dF_dT, dF_dK, d2F_dK2

In [0]:
#Tools functions for neural architecture
def positiveKernelInitializer(shape, 
                              dtype=None, 
                              partition_info=None):
  return tf.abs(tf.keras.initializers.normal()(shape,dtype=dtype, partition_info=partition_info))


#Neural network architecture
def convexLayer1(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    return tf.nn.softplus(layer), layer

def monotonicLayer1(n_units,  tensor, isTraining, name):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor, 
                            units=n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    
    return tf.nn.sigmoid(layer),layer

def convexOutputLayer1(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=2*n_units,
                            kernel_initializer=tf.keras.initializers.glorot_normal(),
                            activation = 'softplus') 
    
     
    layer = tf.layers.dense(layer, 
                            units=1,
                            kernel_initializer=positiveKernelInitializer, 
                            activation = 'softplus')
    
    return layer, layer 
  

def convexLayerHybrid1(n_units, 
                      tensor, 
                      isTraining, 
                      name, 
                      activationFunction2 = Act.softplus,
                      activationFunction1 = Act.exponential,
                      isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=n_units,
                            kernel_initializer=positiveKernelInitializer)
    l1,l2 = tf.split(layer,2,1)
    output = tf.concat([activationFunction1(l1),activationFunction2(l2)],axis=-1)
    return output , layer

def sigmoidGradient(inputTensor):
  return tf.nn.sigmoid(inputTensor) * ( 1 - tf.nn.sigmoid(inputTensor) )

def sigmoidHessian(inputTensor) :
  return (tf.square(1 - tf.nn.sigmoid(inputTensor)) -
          tf.nn.sigmoid(inputTensor) * (1 - tf.nn.sigmoid(inputTensor)))

  
def NNArchitectureConstrainedDupire(n_units, 
                                    strikeTensor,
                                    maturityTensor, 
                                    scaleTensor, 
                                    strikeMinTensor,
                                    vegaRef, 
                                    hyperparameters,
                                    IsTraining=True):
  #First splitted layer
  hidden1S, layer1S = convexLayer1(n_units = n_units,
                                   tensor = strikeTensor,
                                   isTraining=IsTraining,
                                   name = "Hidden1S")
  
  hidden1M,layer1M = monotonicLayer1(n_units = n_units,
                                     tensor = maturityTensor,
                                     isTraining = IsTraining,
                                     name = "Hidden1M")
  
  hidden1 = tf.concat([hidden1S, hidden1M], axis=-1)
  
  #Second layer and output layer
  out, layer = convexOutputLayer1(n_units = n_units,
                                  tensor = hidden1,
                                  isTraining = IsTraining,
                                  name = "Output")
  
  
  dT, dS, HS = exact_derivatives(strikeTensor, maturityTensor)
  
  
  
  #Local volatility
  dupireVol, dupireVar = dupireFormula(HS, dT, 
                                       strikeTensor,
                                       scaleTensor,
                                       strikeMinTensor, 
                                       IsTraining=IsTraining)
  
  #Soft constraints on price
  lambdas = hyperparameters["lambdaSoft"]
  lowerBoundTheta = tf.constant(hyperparameters["lowerBoundTheta"])
  lowerBoundGamma = tf.constant(hyperparameters["lowerBoundGamma"])
  grad_penalty = lambdas * tf.reduce_mean(tf.nn.relu(-dT + lowerBoundTheta) / vegaRef)
  HSScaled = HS / tf.square(scaleTensor)
  hessian_penalty = lambdas * hyperparameters["lambdaGamma"] * tf.reduce_mean(tf.nn.relu(- HSScaled + lowerBoundGamma) / vegaRef)
  
  return out, [out, dupireVol, dT, HSScaled, dupireVar], [grad_penalty, hessian_penalty], evalAndFormatDupireResult


In [0]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [0]:
y_pred2, volLocale2, dNN_T2, gNN_K2, lossSerie2 = create_train_model(NNArchitectureConstrainedDupire,
                                                                     scaledDataSet,
                                                                     False, 
                                                                     hyperparameters,
                                                                     modelName = "convexHybridMatthewDupireVolModel")

In [0]:
plotEpochLoss(lossSerie2)

In [0]:
lossSerie2.iloc[-1]

In [0]:
y_pred2, volLocale2, dNN_T2, gNN_K2, lossSerie2 = create_eval_model(NNArchitectureConstrainedDupire, 
                                                                    scaledDataSet, 
                                                                    False, 
                                                                    hyperparameters,
                                                                    modelName = "convexHybridMatthewDupireVolModel")
modelSummary(y_pred2, volLocale2, dNN_T2, gNN_K2, dataSet)
impV2 = plotImpliedVol(y_pred2, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
volLocale2.loc[(midS0,slice(None))]

In [0]:
y_pred2Test, volLocale2Test, dNN_T2Test, gNN_K2Test, lossSerie2Test = create_eval_model(NNArchitectureConstrainedDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        False, 
                                                                                        hyperparameters,
                                                                                        modelName = "convexHybridMatthewDupireVolModel")
modelSummary(y_pred2Test, volLocale2Test, dNN_T2Test, gNN_K2Test, dataSetTest)
impV2Test = plotImpliedVol(y_pred2Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Hybrid Network (Derivatives from algorithmic differentiation) 

In [0]:

def NNArchitectureConstrainedRawDupire(n_units, 
                                       strikeTensor,
                                       maturityTensor,
                                       scaleTensor,
                                       strikeMinTensor, 
                                       vegaRef, 
                                       hyperparameters,
                                       IsTraining=True):
  #First splitted layer
  hidden1S = convexLayer(n_units = n_units,
                         tensor = strikeTensor,
                         isTraining=IsTraining, 
                         name = "Hidden1S")
  
  hidden1M = monotonicLayer(n_units = n_units,
                            tensor = maturityTensor, 
                            isTraining = IsTraining, 
                            name = "Hidden1M")
  
  hidden1 = tf.concat([hidden1S, hidden1M], axis=-1)
  
  #Second hidden layer and output layer
  out = convexOutputLayer(n_units = n_units,
                          tensor = hidden1,
                          isTraining = IsTraining,
                          name = "Output")
  
  #Compute local volatility
  dupireVol, theta, hK, dupireVar = rawDupireFormula(out, strikeTensor, 
                                                     maturityTensor, 
                                                     scaleTensor, 
                                                     strikeMinTensor,
                                                     IsTraining=IsTraining)

  #Soft constraints for no-arbitrage
  lambdas = hyperparameters["lambdaSoft"] 
  lowerBoundTheta = tf.constant(hyperparameters["lowerBoundTheta"])
  lowerBoundGamma = tf.constant(hyperparameters["lowerBoundGamma"])
  grad_penalty = lambdas * tf.reduce_mean(tf.nn.relu(-theta + lowerBoundTheta) / vegaRef)
  hessian_penalty = lambdas * hyperparameters["lambdaGamma"] * tf.reduce_mean(tf.nn.relu(-hK + lowerBoundGamma) / vegaRef)
  
  return out, [out, dupireVol, theta, hK, dupireVar], [grad_penalty, hessian_penalty], evalAndFormatDupireResult

In [0]:
y_pred3, volLocale3, dNN_T3, gNN_K3, lossSerie3 = create_train_model(NNArchitectureConstrainedRawDupire,
                                                                     scaledDataSet,
                                                                     False, 
                                                                     hyperparameters,
                                                                     modelName = "convexHybridDupireVolModel")

In [0]:
plotEpochLoss(lossSerie3)

In [0]:
lossSerie3.iloc[-1]

In [0]:
y_pred3, volLocale3, dNN_T3, gNN_K3, lossSerie3 = create_eval_model(NNArchitectureConstrainedRawDupire, 
                                                                    scaledDataSet, 
                                                                    False,
                                                                    hyperparameters,
                                                                    modelName = "convexHybridDupireVolModel")
modelSummary(y_pred3, volLocale3, dNN_T3, gNN_K3, dataSet)
impV3 = plotImpliedVol(y_pred3, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
volLocale3.loc[(midS0,slice(None))]

In [0]:
y_pred3Test, volLocale3Test, dNN_T3Test, gNN_K3Test, lossSerie3Test = create_eval_model(NNArchitectureConstrainedRawDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        False, 
                                                                                        hyperparameters,
                                                                                        modelName = "convexHybridDupireVolModel")
modelSummary(y_pred3Test, volLocale3Test, dNN_T3Test, gNN_K3Test, dataSetTest)
impV3Test = plotImpliedVol(y_pred3Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
dNN_T3Test[dNN_T3Test<=0]

### Standard network with soft constraints

In [0]:
def NNArchitectureVanillaSoftDupire(n_units, strikeTensor,
                                    maturityTensor,
                                    scaleTensor,
                                    strikeMinTensor,
                                    vegaRef,
                                    hyperparameters,
                                    IsTraining=True):
  
  inputLayer = tf.concat([strikeTensor,maturityTensor], axis=-1)
  #First layer
  hidden1 = unconstrainedLayer(n_units = n_units,
                               tensor = inputLayer,
                               isTraining=IsTraining, 
                               name = "Hidden1")
  #Second layer
  hidden2 = unconstrainedLayer(n_units = n_units,
                               tensor = hidden1,
                               isTraining=IsTraining, 
                               name = "Hidden2")
  #Output layer
  out = unconstrainedLayer(n_units = 1,
                           tensor = hidden2,
                           isTraining=IsTraining, 
                           name = "Output",
                           activation = None)
  #Local volatility 
  dupireVol, theta, hK, dupireVar = rawDupireFormula(out, strikeTensor,
                                                     maturityTensor,
                                                     scaleTensor,
                                                     strikeMinTensor,
                                                     IsTraining=IsTraining)
  #Soft constraints for no arbitrage
  lambdas = hyperparameters["lambdaSoft"] 
  lowerBoundTheta = tf.constant(hyperparameters["lowerBoundTheta"])
  lowerBoundGamma = tf.constant(hyperparameters["lowerBoundGamma"])
  grad_penalty = lambdas * tf.reduce_mean(tf.nn.relu(-theta + lowerBoundTheta) / vegaRef)
  hessian_penalty = lambdas * hyperparameters["lambdaGamma"] * tf.reduce_mean(tf.nn.relu(-hK + lowerBoundGamma) / vegaRef)
  
  return out, [out, dupireVol, theta, hK, dupireVar], [grad_penalty, hessian_penalty], evalAndFormatDupireResult

In [0]:
y_pred4, volLocale4, dNN_T4, gNN_K4, lossSerie4 = create_train_model(NNArchitectureVanillaSoftDupire,
                                                                     scaledDataSet,
                                                                     False, 
                                                                     hyperparameters,
                                                                     modelName = "convexSoftDupireVolModel")

In [0]:
plotEpochLoss(lossSerie4)

In [0]:
lossSerie4.iloc[-1]

In [0]:
y_pred4, volLocale4, dNN_T4, gNN_K4, lossSerie4 = create_eval_model(NNArchitectureVanillaSoftDupire,
                                                                    scaledDataSet, 
                                                                    False, 
                                                                    hyperparameters,
                                                                    modelName = "convexSoftDupireVolModel")
modelSummary(y_pred4, volLocale4, dNN_T4, gNN_K4, dataSet)
impV4 = plotImpliedVol(y_pred4, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
volLocale4.loc[(midS0,slice(None))]

In [0]:
y_pred4Test, volLocale4Test, dNN_T4Test, gNN_K4Test, lossSerie4Test = create_eval_model(NNArchitectureVanillaSoftDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        False, 
                                                                                        hyperparameters,
                                                                                        modelName = "convexSoftDupireVolModel")
modelSummary(y_pred4Test, volLocale4Test, dNN_T4Test, gNN_K4Test, dataSetTest)
impV4Test = plotImpliedVol(y_pred4Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Unconstrained standard network

In [0]:
def NNArchitectureUnconstrainedDupire(n_units, strikeTensor,
                                      maturityTensor,
                                      scaleTensor,
                                      strikeMinTensor, 
                                      vegaRef,
                                      hyperparameters,
                                      IsTraining=True):
  
  inputLayer = tf.concat([strikeTensor,maturityTensor], axis=-1)
  
  #First layer
  hidden1 = unconstrainedLayer(n_units = n_units,
                               tensor = inputLayer,
                               isTraining=IsTraining, 
                               name = "Hidden1")
  #Second layer
  hidden2 = unconstrainedLayer(n_units = n_units,
                               tensor = hidden1,
                               isTraining=IsTraining, 
                               name = "Hidden2")
  #Ouput layer
  out = unconstrainedLayer(n_units = 1,
                           tensor = hidden2,
                           isTraining=IsTraining, 
                           name = "Output",
                           activation = None)
  #Local volatility
  dupireVol, theta, hK, dupireVar = rawDupireFormula(out, strikeTensor,
                                                     maturityTensor,
                                                     scaleTensor,
                                                     strikeMinTensor,
                                                     IsTraining=IsTraining)
  
  return out, [out, dupireVol, theta, hK, dupireVar], [], evalAndFormatDupireResult

In [0]:
y_pred5, volLocale5, dNN_T5, gNN_K5, lossSerie5 = create_train_model(NNArchitectureUnconstrainedDupire,
                                                                     scaledDataSet,
                                                                     False, 
                                                                     hyperparameters,
                                                                     modelName = "unconstrainedDupireVolModel")

In [0]:
plotEpochLoss(lossSerie5)

In [0]:
lossSerie5.iloc[-1]

In [0]:
y_pred5, volLocale5, dNN_T5, gNN_K5, lossSerie5 = create_eval_model(NNArchitectureUnconstrainedDupire,
                                                                    scaledDataSet,
                                                                    False,
                                                                    hyperparameters,
                                                                    modelName = "unconstrainedDupireVolModel")
modelSummary(y_pred5, volLocale5, dNN_T5, gNN_K5, dataSet)
impV5 = plotImpliedVol(y_pred5, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
volLocale5.loc[(midS0,slice(None))]

In [0]:
y_pred5Test, volLocale5Test, dNN_T5Test, gNN_K5Test, lossSerie5Test = create_eval_model(NNArchitectureUnconstrainedDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        False, 
                                                                                        hyperparameters,
                                                                                        modelName = "unconstrainedDupireVolModel")
modelSummary(y_pred5Test, volLocale5Test, dNN_T5Test, gNN_K5Test, dataSetTest)
impV5Test = plotImpliedVol(y_pred5Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Hard constrained architecture

In [0]:
#Tools functions for hard constrained neural architecture

def convexLayerHard(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=n_units,
                            kernel_constraint = tf.keras.constraints.NonNeg(), 
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    return tf.nn.softplus(layer), layer 

def monotonicLayerHard(n_units,  tensor, isTraining, name):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor, 
                            units=n_units,
                            kernel_constraint = tf.keras.constraints.NonNeg(), 
                            kernel_initializer=tf.keras.initializers.glorot_normal())
    
    
    
    return tf.nn.sigmoid(layer),layer

def convexOutputLayerHard(n_units, tensor, isTraining, name, isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=2*n_units,
                            kernel_constraint = tf.keras.constraints.NonNeg(), 
                            kernel_initializer=tf.keras.initializers.glorot_normal(),
                            activation = 'softplus') 
    
     
    layer = tf.layers.dense(layer, 
                            units=1,
                            kernel_constraint = tf.keras.constraints.NonNeg(), 
                            kernel_initializer=positiveKernelInitializer, 
                            activation = 'softplus')
    
    return layer, layer 
  

def convexLayerHybridHard(n_units,
                          tensor,
                          isTraining,
                          name,
                          activationFunction2 = Act.softplus,
                          activationFunction1 = Act.exponential,
                          isNonDecreasing = True):
  with tf.name_scope(name):
    layer = tf.layers.dense(tensor if isNonDecreasing else (- tensor), 
                            units=n_units,
                            kernel_constraint = tf.keras.constraints.NonNeg(), 
                            kernel_initializer=positiveKernelInitializer)
    l1,l2 = tf.split(layer,2,1)
    output = tf.concat([activationFunction1(l1),activationFunction2(l2)],axis=-1)
    return output , layer

def sigmoidGradientHard(inputTensor):
  return tf.nn.sigmoid(inputTensor) * ( 1 - tf.nn.sigmoid(inputTensor) )

def sigmoidHessianHard(inputTensor) :
  return (tf.square(1 - tf.nn.sigmoid(inputTensor)) -
          tf.nn.sigmoid(inputTensor) * (1 - tf.nn.sigmoid(inputTensor)))
  


  
def NNArchitectureHardConstrainedDupire(n_units, strikeTensor, 
                                        maturityTensor,
                                        scaleTensor,
                                        strikeMinTensor, 
                                        vegaRef,
                                        hyperparameters,
                                        IsTraining=True):
  #First layer
  hidden1S, layer1S = convexLayerHard(n_units = n_units,
                                      tensor = strikeTensor,
                                      isTraining=IsTraining,
                                      name = "Hidden1S")
  
  hidden1M,layer1M = monotonicLayerHard(n_units = n_units,
                                        tensor = maturityTensor,
                                        isTraining = IsTraining,
                                        name = "Hidden1M")
  
  hidden1 = tf.concat([hidden1S, hidden1M], axis=-1)
  
  #Second layer and output layer
  out, layer = convexOutputLayerHard(n_units = n_units,
                                     tensor = hidden1,
                                     isTraining = IsTraining,
                                     name = "Output")
  #Local volatility
  dupireVol, theta, hK, dupireVar = rawDupireFormula(out, strikeTensor,
                                                     maturityTensor,
                                                     scaleTensor,
                                                     strikeMinTensor,
                                                     IsTraining=IsTraining)
  
  return out, [out, dupireVol, theta, hK, dupireVar], [], evalAndFormatDupireResult

In [0]:
y_pred6, volLocale6, dNN_T6, gNN_K6, lossSerie6 = create_train_model(NNArchitectureHardConstrainedDupire,
                                                                     scaledDataSet,
                                                                     False, 
                                                                     hyperparameters,
                                                                     modelName = "convexHardDupireVolModel")

In [0]:
plotEpochLoss(lossSerie6)

In [0]:
lossSerie6.iloc[-1]

In [0]:
y_pred6, volLocale6, dNN_T6, gNN_K6, lossSerie6 = create_eval_model(NNArchitectureHardConstrainedDupire, 
                                                                    scaledDataSet, 
                                                                    False, 
                                                                    hyperparameters,
                                                                    modelName = "convexHardDupireVolModel")
modelSummary(y_pred6, volLocale6, dNN_T6, gNN_K6, dataSet)
impV6 = plotImpliedVol(y_pred6, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
volLocale6.loc[(midS0,slice(None))]

In [0]:
y_pred6Test, volLocale6Test, dNN_T6Test, gNN_K6Test, lossSerie6Test = create_eval_model(NNArchitectureHardConstrainedDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        False, 
                                                                                        hyperparameters,
                                                                                        modelName = "convexHardDupireVolModel")
modelSummary(y_pred6Test, volLocale6Test, dNN_T6Test, gNN_K6Test, dataSetTest)
impV6Test = plotImpliedVol(y_pred6Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

## Dupire regularization 

Same lines as above except that dupire regularization is now activated.

### Hybrid architecture (Exact derivatives)

In [0]:
y_pred8, volLocale8, dNN_T8, gNN_K8, lossSerie8 = create_train_model(NNArchitectureConstrainedDupire,
                                                                     scaledDataSet,
                                                                     True, 
                                                                     hyperparameters,
                                                                     modelName = "regularizedConvexHybridMatthewDupireVolModel")

In [0]:
plotEpochLoss(lossSerie8)

In [0]:
lossSerie8.iloc[-1]

In [0]:
y_pred8, volLocale8, dNN_T8, gNN_K8, lossSerie8 = create_eval_model(NNArchitectureConstrainedDupire, 
                                                                    scaledDataSet, 
                                                                    True, 
                                                                    hyperparameters,
                                                                    modelName = "regularizedConvexHybridMatthewDupireVolModel")
modelSummary(y_pred8, volLocale8, dNN_T8, gNN_K8, dataSet)
impV8 = plotImpliedVol(y_pred8, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred8Test, volLocale8Test, dNN_T8Test, gNN_K8Test, lossSerie8Test = create_eval_model(NNArchitectureConstrainedDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        True, 
                                                                                        hyperparameters,
                                                                                        modelName = "regularizedConvexHybridMatthewDupireVolModel")
modelSummary(y_pred8Test, volLocale8Test, dNN_T8Test, gNN_K8Test, dataSetTest)
impV8Test = plotImpliedVol(y_pred8Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Unconstrained standard network

In [0]:
y_pred9, volLocale9, dNN_T9, gNN_K9, lossSerie9 = create_train_model(NNArchitectureUnconstrainedDupire,
                                                                     scaledDataSet,
                                                                     True, 
                                                                     hyperparameters,
                                                                     modelName = "regularizedUnconstrainedDupireVolModel")

In [0]:
plotEpochLoss(lossSerie9)

In [0]:
lossSerie9.iloc[-1]

In [0]:
y_pred9, volLocale9, dNN_T9, gNN_K9, lossSerie9 = create_eval_model(NNArchitectureUnconstrainedDupire, 
                                                                    scaledDataSet, 
                                                                    True, 
                                                                    hyperparameters,
                                                                    modelName = "regularizedUnconstrainedDupireVolModel")
modelSummary(y_pred9, volLocale9, dNN_T9, gNN_K9, dataSet)
impV9 = plotImpliedVol(y_pred9, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred9Test, volLocale9Test, dNN_T9Test, gNN_K9Test, lossSerie9Test = create_eval_model(NNArchitectureUnconstrainedDupire, 
                                                                                        scaledDataSetTest, 
                                                                                        True, 
                                                                                        hyperparameters,
                                                                                        modelName = "regularizedUnconstrainedDupireVolModel")
modelSummary(y_pred9Test, volLocale9Test, dNN_T9Test, gNN_K9Test, dataSetTest)
impV9Test = plotImpliedVol(y_pred9Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Hybrid Network (Derivatives from algorithmic differentiation) 

In [0]:
y_pred10, volLocale10, dNN_T10, gNN_K10, lossSerie10 = create_train_model(NNArchitectureConstrainedRawDupire,
                                                                          scaledDataSet,
                                                                          True,
                                                                          hyperparameters,
                                                                          modelName = "regularizedConvexHybridDupireVolModel")

In [0]:
plotEpochLoss(lossSerie10)

In [0]:
lossSerie10.iloc[-1]

In [0]:
y_pred10, volLocale10, dNN_T10, gNN_K10, lossSerie10 = create_eval_model(NNArchitectureConstrainedRawDupire,
                                                                         scaledDataSet,
                                                                         True,
                                                                         hyperparameters,
                                                                         modelName = "regularizedConvexHybridDupireVolModel")
modelSummary(y_pred10, volLocale10, dNN_T10, gNN_K10, dataSet)
impV10 = plotImpliedVol(y_pred10, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred10Test, volLocale10Test, dNN_T10Test, gNN_K10Test, lossSerie10Test = create_eval_model(NNArchitectureConstrainedRawDupire,
                                                                                             scaledDataSetTest,
                                                                                             True,
                                                                                             hyperparameters,
                                                                                             modelName = "regularizedConvexHybridDupireVolModel")
modelSummary(y_pred10Test, volLocale10Test, dNN_T10Test, gNN_K10Test, dataSetTest)
impV10Test = plotImpliedVol(y_pred10Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Standard network with soft constraints

In [0]:
y_pred11, volLocale11, dNN_T11, gNN_K11, lossSerie11 = create_train_model(NNArchitectureVanillaSoftDupire,
                                                                          scaledDataSet,
                                                                          True,
                                                                          hyperparameters,
                                                                          modelName = "regularizedConvexSoftDupireVolModel")

In [0]:
plotEpochLoss(lossSerie11)

In [0]:
lossSerie11.iloc[-1]

In [0]:
y_pred11, volLocale11, dNN_T11, gNN_K11, lossSerie11 = create_eval_model(NNArchitectureVanillaSoftDupire,
                                                                         scaledDataSet,
                                                                         True,
                                                                         hyperparameters,
                                                                         modelName = "regularizedConvexSoftDupireVolModel")
modelSummary(y_pred11, volLocale11, dNN_T11, gNN_K11, dataSet)
impV11 = plotImpliedVol(y_pred11, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred11Test, volLocale11Test, dNN_T11Test, gNN_K11Test, lossSerie11Test = create_eval_model(NNArchitectureVanillaSoftDupire,
                                                                                             scaledDataSetTest,
                                                                                             True,
                                                                                             hyperparameters,
                                                                                             modelName = "regularizedConvexSoftDupireVolModel")
modelSummary(y_pred11Test, volLocale11Test, dNN_T11Test, gNN_K11Test, dataSetTest)
impV11Test = plotImpliedVol(y_pred11Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

### Hard constrained architecture

In [0]:
y_pred12, volLocale12, dNN_T12, gNN_K12, lossSerie12 = create_train_model(NNArchitectureHardConstrainedDupire,
                                                                          scaledDataSet,
                                                                          True,
                                                                          hyperparameters,
                                                                          modelName = "regularizedConvexHardDupireVolModel")

In [0]:
plotEpochLoss(lossSerie12)

In [0]:
lossSerie12.iloc[-1]

In [0]:
y_pred12, volLocale12, dNN_T12, gNN_K12, lossSerie12 = create_eval_model(NNArchitectureHardConstrainedDupire,
                                                                         scaledDataSet,
                                                                         True,
                                                                         hyperparameters,
                                                                         modelName = "regularizedConvexHardDupireVolModel")
modelSummary(y_pred12, volLocale12, dNN_T12, gNN_K12, dataSet)
impV12 = plotImpliedVol(y_pred12, dataSet["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

In [0]:
y_pred12Test, volLocale12Test, dNN_T12Test, gNN_K12Test, lossSerie12Test = create_eval_model(NNArchitectureHardConstrainedDupire,
                                                                                             scaledDataSetTest,
                                                                                             True,
                                                                                             hyperparameters,
                                                                                             modelName = "regularizedConvexHardDupireVolModel")
modelSummary(y_pred12Test, volLocale12Test, dNN_T12Test, gNN_K12Test, dataSetTest)
impV12Test = plotImpliedVol(y_pred12Test, dataSetTest["ImpliedVol"], rIntegralSpline=riskFreeIntegral, qIntegralSpline=divSpreadIntegral)

## Monte Carlo pricing

### Monte Carlo with implied vol

In [0]:
nbTimeStep = 100
nbPaths = 100000
def MonteCarloPricerImplicit(S,
                             Strike,
                             Maturity,
                             rSpline,
                             divSpline,
                             nbPaths,
                             nbTimeStep,
                             impliedVol):
  time_grid = np.linspace(0, Maturity, int(nbTimeStep + 1))
  timeStep = Maturity / nbTimeStep
  gaussianNoise = np.random.normal(scale = np.sqrt(timeStep), size=(nbTimeStep, nbPaths))

  logReturn = np.zeros((nbTimeStep + 1, nbPaths))
  logReturn[0,:] = 0

  for i in range(nbTimeStep) :
      t = time_grid[i]

      St = S0 * np.exp(logReturn[i,:])
      volLocale = impliedVol

      mu = rSpline(t) - divSpline(t)
      drift = np.ones(nbPaths) * (mu - np.square(volLocale) / 2.0) 
      logReturn[i + 1, :] = logReturn[i,:] + drift * timeStep + gaussianNoise[i,:] * volLocale
  SFinal = S0 * np.exp(logReturn[-1, :])
  return np.mean(np.maximum(Strike - SFinal, 0))

def MonteCarloPricerVectorizedImplicit(S,
                                       dataSet,
                                       rSpline,
                                       divSpline,
                                       nbPaths,
                                       nbTimeStep):
  func = lambda x : MonteCarloPricerImplicit(S, x["Strike"], x["Maturity"], riskCurvespline, divSpline, nbPaths, nbTimeStep, x["ImpliedVol"])
  return dataSet.apply(func, axis=1) * np.exp(-riskFreeIntegral(dataSet.index.get_level_values("Maturity")))

In [0]:
mcResRef = MonteCarloPricerVectorizedImplicit(S0[0],
                                              dataSet,
                                              riskCurvespline,
                                              divSpline,
                                              nbPaths,
                                              nbTimeStep)
mcResRef.head()

In [0]:
predictionDiagnosis(mcResRef, dataSet["Price"], " Price ", yMin=4100)

### Monte Carlo local volatility

#### Constant local volatility

In [0]:
def volLocaleTest(S, T):
  return np.ones_like(S) * 0.23

In [0]:
nbTimeStep = 100
nbPaths = 100000
def MonteCarloPricer(S, 
                     Strike, 
                     Maturity, 
                     rSpline, 
                     divSpline, 
                     nbPaths, 
                     nbTimeStep, 
                     volLocaleFunction):
  time_grid = np.linspace(0, Maturity, int(nbTimeStep + 1))
  timeStep = Maturity / nbTimeStep
  gaussianNoise = np.random.normal(scale = np.sqrt(timeStep), size=(nbTimeStep, nbPaths))

  logReturn = np.zeros((nbTimeStep + 1, nbPaths))
  logReturn[0,:] = 0

  for i in range(nbTimeStep) :
      t = time_grid[i]

      St = S0 * np.exp(logReturn[i,:])
      volLocale = volLocaleFunction(St, np.ones(nbPaths) * t)

      mu = rSpline(t) - divSpline(t)
      drift = np.ones(nbPaths) * (mu - np.square(volLocale) / 2.0) 
      logReturn[i + 1, :] = logReturn[i,:] + drift * timeStep + gaussianNoise[i,:] * volLocale
  SFinal = S0 * np.exp(logReturn[-1, :])
  return np.mean(np.maximum(Strike - SFinal, 0))

def MonteCarloPricerVectorized(S, 
                               dataSet,
                               rSpline, 
                               divSpline, 
                               nbPaths, 
                               nbTimeStep, 
                               volLocaleFunction):
  func = lambda x : MonteCarloPricer(S, x["Strike"], x["Maturity"], riskCurvespline, divSpline, nbPaths, nbTimeStep, volLocaleFunction)
  return dataSet.apply(func, axis=1) * np.exp(-riskFreeIntegral(dataSet.index.get_level_values("Maturity")))

In [0]:
mcResSigmaRef = MonteCarloPricerVectorized(S0[0],
                                           dataSet,
                                           riskCurvespline,
                                           divSpline,
                                           nbPaths,
                                           nbTimeStep,
                                           volLocaleTest)
mcResSigmaRef.head()

In [0]:
(mcResSigmaRef <= dataSet["Price"]).sum()

In [0]:
predictionDiagnosis(mcResSigmaRef, dataSet["Price"], " Price ", yMin=4100)

#### Extracting neural local volatility





In [0]:
def evalVolLocale(NNFactory,
                  strikes,
                  maturities,
                  dataSet,
                  hyperParameters,
                  modelName = "bestModel"):
    
    hidden_nodes = hyperParameters["nbUnits"] 

    # Reset the graph
    tf.reset_default_graph()
    
    # Placeholders for input and output data   
    Strike = tf.placeholder(tf.float32,[None,1])
    Maturity = tf.placeholder(tf.float32,[None,1])
    factorPrice = tf.placeholder(tf.float32,[None,1])
    y = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y')
    vegaRef = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='vegaRef')
    learningRateTensor = tf.placeholder(tf.float32,[])
    
    #Get scaling for strike
    colStrikeIndex = dataSet.columns.get_loc("ChangedStrike")
    maxColFunction = scaler.data_max_[colStrikeIndex]
    minColFunction = scaler.data_min_[colStrikeIndex]
    scF = (maxColFunction - minColFunction) 
    scaleTensor = tf.constant(scF, dtype=tf.float32)
    strikeMinTensor = tf.constant(minColFunction, dtype=tf.float32)

    price_pred_tensor = None
    TensorList = None
    penalizationList = None 
    formattingFunction = None
    price_pred_tensor, TensorList, penalizationList, formattingFunction = NNFactory(hidden_nodes,
                                                                                    Strike,
                                                                                    Maturity,
                                                                                    scaleTensor,
                                                                                    strikeMinTensor,
                                                                                    vegaRef,
                                                                                    hyperParameters,
                                                                                    IsTraining=False)# one hidden layer


    price_pred_tensor_sc= tf.multiply(factorPrice,price_pred_tensor)
    TensorList[0] = price_pred_tensor_sc
    
    # Define a loss function
    pointwiseError = tf.reduce_mean(tf.abs(price_pred_tensor_sc - y) / vegaRef)
    errors = tf.add_n([pointwiseError] + penalizationList) 
    loss = tf.log(tf.reduce_mean(errors))

    optimizer = tf.train.AdamOptimizer(learning_rate=learningRateTensor)
    train = optimizer.minimize(loss)

    # Initialize variables and run session
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(init)
    n = strikes.shape[0]
    changedVar = changeOfVariable(strikes, maturities)
    scaledStrike = (changedVar[0]-minColFunction)/scF
    dividendFactor = changedVar[1]

    def createFeedDict(s, t, d):
        batchSize = s.shape[0]
        feedDict = {Strike : np.reshape(s, (batchSize,1)), 
                    Maturity : np.reshape(t, (batchSize,1)) ,  
                    factorPrice : np.reshape(d, (batchSize,1)), 
                    vegaRef : np.ones((batchSize,1))}
        return feedDict
    
    epochFeedDict = createFeedDict(scaledStrike, maturities, dividendFactor)
    
    saver.restore(sess, modelName)  

    evalList = sess.run(TensorList, feed_dict=epochFeedDict)
    
    sess.close()
    
    return pd.Series(evalList[1].flatten(), index = pd.MultiIndex.from_arrays([strikes, maturities], names=('Strike', 'Maturity')))





In [0]:
strikeLow = dataSetTest["Strike"].min()
strikeUp = dataSetTest["Strike"].max()
strikeGrid = np.linspace(strikeLow, strikeUp, 1000)
matLow = dataSetTest["Maturity"].min()
matUp = dataSetTest["Maturity"].max()
matGrid = np.linspace(matLow, matUp, 1000)
volLocaleGrid = np.meshgrid(strikeGrid, matGrid)

In [0]:
def interpolatedMCLocalVolatility(localVol, strikes, maturities):
    strikeLocVol = np.ravel(localVol.index.get_level_values("Strike").values)
    maturityLocVol = np.ravel(localVol.index.get_level_values("Maturity").values)
    xym = np.vstack((strikeLocVol, maturityLocVol)).T
    opts = {'balanced_tree': False, 'compact_nodes': False}
    f =  interpolate.NearestNDInterpolator(xym,
                                           localVol.values.flatten(), 
                                           rescale=True,
                                           tree_options=opts)

    strikePrice = strikes
    maturityPrice = maturities
    #func = lambda x : f(x[0],x[1])
    coordinates =  np.array( f(strikePrice, maturityPrice) ).flatten() # np.array( list( map( func, zip(strikePrice, maturityPrice) ) ) ).flatten() 
    return pd.Series(coordinates, index = pd.MultiIndex.from_arrays([strikes, maturities], names=('Strike', 'Maturity')))



##### Standard network, soft constraints

In [0]:
def neuralVolLocale(s,t):
  vLoc = evalVolLocale(NNArchitectureVanillaSoftDupire,
                       s, t,
                       dataSet,
                       hyperparameters,
                       modelName = "regularizedConvexSoftDupireVolModel")
  return vLoc

In [0]:
volLocalInterp = neuralVolLocale(volLocaleGrid[0].flatten(), 
                                 volLocaleGrid[1].flatten())
volLocalInterp.head()

In [0]:
volLocalInterp2 = neuralVolLocale(dataSetTest.index.get_level_values("Strike").values.flatten(), 
                                  dataSetTest.index.get_level_values("Maturity").values.flatten())
volLocalInterp2.head()

In [0]:
nnVolLocale = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp, x, y)

In [0]:
nnVolLocale2 = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp2, x, y)

In [0]:
plotSerie(volLocalInterp,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

In [0]:
plotSerie(volLocalInterp2,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

In [0]:
plotSerie(dataSetTest["locvol"],
          Title = 'Inteprolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

##### Hard constraint Regularized

In [0]:
def neuralVolLocaleHardReg(s,t):
  vLoc = evalVolLocale(NNArchitectureHardConstrainedDupire,
                       s, t,
                       dataSet,
                       hyperparameters,
                       modelName = "regularizedConvexHardDupireVolModel")
  return vLoc

In [0]:
volLocalInterp3 = neuralVolLocaleHardReg(volLocaleGrid[0].flatten(),
                                         volLocaleGrid[1].flatten())
volLocalInterp3.head()

In [0]:
volLocalInterp4 = neuralVolLocaleHardReg(dataSetTest.index.get_level_values("Strike").values.flatten(),
                                         dataSetTest.index.get_level_values("Maturity").values.flatten())
volLocalInterp4.head()

In [0]:
nnVolLocale3 = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp3, x, y)

In [0]:
nnVolLocale4 = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp4, x, y)

In [0]:
plotSerie(volLocalInterp3,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

In [0]:
plotSerie(volLocalInterp4,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

##### Hard constraint

In [0]:
def neuralVolLocaleHard(s,t):
  vLoc = evalVolLocale(NNArchitectureHardConstrainedDupire,
                       s, t,
                       dataSet,
                       hyperparameters,
                       modelName = "convexHardDupireVolModel")
  return vLoc

In [0]:
volLocalInterp5 = neuralVolLocaleHard(volLocaleGrid[0].flatten(),
                                      volLocaleGrid[1].flatten())
volLocalInterp5.head()

In [0]:
volLocalInterp6 = neuralVolLocaleHard(dataSetTest.index.get_level_values("Strike").values.flatten(),
                                      dataSetTest.index.get_level_values("Maturity").values.flatten())
volLocalInterp6.head()

In [0]:
nnVolLocale5 = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp5, x, y)

In [0]:
nnVolLocale6 = lambda x,y : interpolatedMCLocalVolatility(volLocalInterp6, x, y)

In [0]:
plotSerie(volLocalInterp5,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

In [0]:
plotSerie(volLocalInterp6,
          Title = 'Interpolated Local Volatility Surface',
          az=30,
          yMin=0.0*S0,
          yMax=2.0*S0, 
          zAsPercent=True)

#### Tikhonov local volatility

In [0]:
def interpolatedMCTikhonov(localVol, strikes, maturities):
    strikeLocVol = np.ravel(localVol.index.get_level_values("Strike").values)
    maturityLocVol = np.ravel(localVol.index.get_level_values("Maturity").values)
    xym = np.vstack((strikeLocVol, maturityLocVol)).T
    opts = {'balanced_tree': False, 'compact_nodes': False}
    f =  interpolate.NearestNDInterpolator(xym,
                                           localVol.values.flatten(), 
                                           rescale=True,
                                           tree_options=opts)

    strikePrice = strikes
    maturityPrice = maturities
    #func = lambda x : f(x[0],x[1])
    coordinates =  np.array( f(strikePrice, maturityPrice) ).flatten() # np.array( list( map( func, zip(strikePrice, maturityPrice) ) ) ).flatten() 
    return pd.Series(coordinates, index = pd.MultiIndex.from_arrays([strikes, maturities], names=('Strike', 'Maturity')))

nnTikhonov = lambda x,y : interpolatedMCTikhonov(dataSetTest["locvol"], x, y)

In [0]:
mcResTikhonov = MonteCarloPricerVectorized(S0[0],
                                           dataSet,
                                           riskCurvespline,
                                           divSpline,
                                           nbPaths,
                                           nbTimeStep,
                                           nnTikhonov)
mcResTikhonov.head()

In [0]:
predictionDiagnosis(mcResTikhonov, dataSet["Price"], " Price ", yMin=4100)

#### Neural local Volatility

##### Standard Network soft constraint

In [0]:
mcResVolLocale = MonteCarloPricerVectorized(S0[0],
                                            dataSet,
                                            riskCurvespline,
                                            divSpline,
                                            nbPaths,
                                            nbTimeStep,
                                            nnVolLocale)
mcResVolLocale.head()

In [0]:
predictionDiagnosis(mcResVolLocale, dataSet["Price"], " Price ", yMin=4100)

In [0]:
dataSet.tail()

In [0]:
mcResVolLocale2 = MonteCarloPricerVectorized(S0[0],
                                            dataSet,
                                            riskCurvespline,
                                            divSpline,
                                            nbPaths,
                                            nbTimeStep,
                                            nnVolLocale2)
mcResVolLocale2.head()

In [0]:
predictionDiagnosis(mcResVolLocale2, dataSet["Price"], " Price ", yMin=4100)

##### Hard constraint Regularized

In [0]:
mcResVolLocale3 = MonteCarloPricerVectorized(S0[0],
                                             dataSet,
                                             riskCurvespline,
                                             divSpline,
                                             nbPaths,
                                             nbTimeStep,
                                             nnVolLocale3)
mcResVolLocale3.head()

In [0]:
predictionDiagnosis(mcResVolLocale3, dataSet["Price"], " Price ", yMin=4100)

In [0]:
mcResVolLocale4 = MonteCarloPricerVectorized(S0[0],
                                             dataSet,
                                             riskCurvespline,
                                             divSpline,
                                             nbPaths,
                                             nbTimeStep,
                                             nnVolLocale4)
mcResVolLocale4.head()

In [0]:
predictionDiagnosis(mcResVolLocale4, dataSet["Price"], " Price ", yMin=4100)

##### Hard constraint

In [0]:
mcResVolLocale5 = MonteCarloPricerVectorized(S0[0],
                                             dataSet,
                                             riskCurvespline,
                                             divSpline,
                                             nbPaths,
                                             nbTimeStep,
                                             nnVolLocale5)
mcResVolLocale5.head()

In [0]:
predictionDiagnosis(mcResVolLocale5, dataSet["Price"], " Price ", yMin=4100)

In [0]:
mcResVolLocale6 = MonteCarloPricerVectorized(S0[0],
                                             dataSet,
                                             riskCurvespline,
                                             divSpline,
                                             nbPaths,
                                             nbTimeStep,
                                             nnVolLocale6)
mcResVolLocale6.head()

In [0]:
predictionDiagnosis(mcResVolLocale6, dataSet["Price"], " Price ", yMin=4100)

## Hyperparameter selection

In [0]:
def selectHyperparameters(hyperparameters, parameterOfInterest, modelFactory, modelName, activateDupireReg, logGrid = True):
    oldValue = hyperparameters[parameterOfInterest]
    gridValue = oldValue * ( np.exp( np.log(10) * np.array([-2,-1, 0, 1, 2])) if logGrid else np.array([0.2, 0.5, 1, 2, 5]) )
    
    oldNbEpochs = hyperparameters["maxEpoch"]
    hyperparameters["maxEpoch"] = int(oldNbEpochs / 10)
    trainLoss = {}
    arbitrageViolation = {}
    for v in gridValue :
        hyperparameters[parameterOfInterest] = int(v)
        pred, volLoc, theta, gammaK, loss = create_train_model(modelFactory,
                                                               scaledDataSet,
                                                               activateDupireReg,
                                                               hyperparameters,
                                                               modelName = modelName)
        nbArbitrageViolation = np.sum((theta <= 0)) + np.sum((gammaK <= 0))
        trainLoss[v] = min(loss)
        arbitrageViolation[v] = nbArbitrageViolation
        print()
        print()

    hyperparameters["maxEpoch"] = oldNbEpochs
    hyperparameters[parameterOfInterest] = oldValue
    # Plot curves
    
    fig, ax1 = plt.subplots()
    if logGrid :
        plt.xscale('symlog')
    
    color = 'tab:red'
    ax1.set_xlabel('Value')
    ax1.set_ylabel('Loss', color=color)
    ax1.plot(pd.Series(trainLoss), color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    
    color = 'tab:blue'
    ax2.set_ylabel('Arbitrage violation', color=color)  # we already handled the x-label with ax1
    ax2.plot(pd.Series(arbitrageViolation), color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    
    return

In [0]:
def selectHyperparametersRandom(hyperparameters, 
                                parametersOfInterest, 
                                modelFactory, 
                                modelName, 
                                activateDupireReg, 
                                nbAttempts,
                                logGrid = True):
    oldValue = {} 
    for k in parametersOfInterest :
        oldValue[k] = hyperparameters[k]
    
    gridValue = np.exp( np.log(10) * np.array([-2,-1, 0, 1, 2])) if logGrid else np.array([0.2, 0.5, 1, 2, 5]) 
    
    oldNbEpochs = hyperparameters["maxEpoch"]
    hyperparameters["maxEpoch"] = int(oldNbEpochs / 10)
    trainLoss = {}
    arbitrageViolation = {}
    nbTry = nbAttempts
    for v in range(nbTry) :
        combination = np.random.randint(5, size = len(parametersOfInterest) )
        for p in range(len(parametersOfInterest)):
            hyperparameters[parametersOfInterest[p]] = oldValue[parametersOfInterest[p]] * gridValue[int(combination[p])]
            print(parametersOfInterest[p] , " : ", hyperparameters[parametersOfInterest[p]])
        pred, volLoc, theta, gammaK, loss = create_train_model(modelFactory,
                                                               scaledDataSet,
                                                               activateDupireReg,
                                                               hyperparameters,
                                                               modelName = modelName)
        nbArbitrageViolation = np.sum((theta <= 0)) + np.sum((gammaK <= 0))
        print("loss : ", min(loss))
        print("nbArbitrageViolation : ", nbArbitrageViolation)
        print()
        print()
        print()

    hyperparameters["maxEpoch"] = oldNbEpochs
    for k in parametersOfInterest :
        hyperparameters[k] = oldValue[k]
    
    return

In [0]:
selectHyperparametersRandom(hyperparameters,
                            ["lambdaLocVol","lambdaSoft","lambdaGamma"],
                            NNArchitectureConstrainedRawDupire,
                            "hyperParameters",
                            True, 
                            100,
                            logGrid = True)

In [0]:

hyperparameters["lambdaLocVol"] = 100
hyperparameters["lambdaSoft"] = 100 
hyperparameters["lambdaGamma"] = 10000

In [0]:
selectHyperparameters(hyperparameters, 
                      "lambdaLocVol", 
                      NNArchitectureVanillaSoftDupire, 
                      "hyperParameters", 
                      True, 
                      logGrid = True)

In [0]:
selectHyperparameters(hyperparameters, 
                      "DupireVarCap", 
                      NNArchitectureConstrainedRawDupire, 
                      "hyperParameters", 
                      True, 
                      logGrid = True)

In [0]:
selectHyperparameters(hyperparameters, 
                      "lambdaLocVol", 
                      NNArchitectureUnconstrainedDupire, 
                      "hyperParameters", 
                      True, 
                      logGrid = True)

In [0]:
hyperparameters["lambdaLocVol"] = 100

In [0]:
selectHyperparameters(hyperparameters, 
                      "lambdaLocVol", 
                      NNArchitectureConstrainedRawDupire, 
                      "hyperParameters", 
                      True, 
                      logGrid = True)

In [0]:
hyperparameters["nbUnits"] = 40

In [0]:
selectHyperparameters(hyperparameters, 
                      "nbUnits", 
                      NNArchitectureVanillaSoftDupire, 
                      "hyperParameters", 
                      True, 
                      logGrid = False)

In [0]:
hyperparameters["nbUnits"] = 200